In [ ]:
import natsort
import numpy as np
import pandas as pd
import param
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import re
import traceback
import panel as pn
pn.extension("plotly")
import io
from io import BytesIO
from io import StringIO
from bokeh.models.widgets.tables import NumberFormatter
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise as pw
import json
import ast
import statistics 
import matplotlib.pyplot as plt
import matplotlib_venn as venn
from matplotlib_venn import venn2, venn3, venn3_circles
from PIL import Image
from upsetplot import from_memberships
from upsetplot import plot
pn.extension()

In [145]:
exec(open("class.py").read())

i_file = pn.widgets.FileInput(name="Upload file")

loading_status = pn.Row()
idle = pn.indicators.LoadingSpinner(value=False, width=100, height=100, color="primary")
loading = pn.indicators.LoadingSpinner(value=True, width=100, height=100, color="primary")

button_analysis = pn.widgets.Button(name="Analyse clusters", width=50)

i_logOR01_selection = pn.widgets.Select(options=["0/1 normalized data", "log transformed data", "stringency filtered raw data"], name="Select type of data for download", width=300)

i_acquisition = pn.widgets.Select(options=["LFQ6 - Spectronaut", "LFQ5 - Spectronaut", "LFQ6 - MQ", "LFQ5 - MQ",  "SILAC - MQ"], name="Acquisition", width=300)
i_organism = pn.widgets.Select(options=list(SpatialDataSet.markerproteins_set.keys()), name="Organism", width=300) #


i_comment = pn.widgets.input.TextAreaInput(name="Additional Comments", placeholder="Write any kind of information assoiciated with this dataset here...")

i_clusterwidget = pn.widgets.Select(options=["Proteasome", "Lysosome"], name="Cluster of interest", width=300)
i_mapwidget = pn.widgets.Select(options=["Map1", "Map2"], name="Map of interest", width=300)

i_collapse_maps_PCA = pn.widgets.Checkbox(value=False, name="Collapse maps")

cache_uploaded = pn.widgets.Checkbox(value=False)


cache_run = pn.widgets.Checkbox(value=False)

analysis_status = pn.Pane("No analysis run yet", width=1000)
filereading_status = pn.Pane("No data import yet", width=1000)

i_expname = pn.widgets.TextInput(name="Experiment Name", placeholder="Enter your experiment name here here...")

i_consecutiveLFQi = pn.widgets.IntSlider(name="Consecutive LFQ intensities", start=1, end=10, step=1, value=4)
i_summed_MSMS_counts = pn.widgets.IntSlider(name="Summed MS/MS counts ≥ 2n x number of fractions", start=1, end=10, step=1, value=2)

i_RatioHLcount = pn.widgets.IntSlider(name="Quantification events (Ratio H/L Count)", start=1, end=10, step=1, value=2)
i_RatioVariability = pn.widgets.IntSlider(name="Ratio H/L variability [%]", start=0, end=100, step=5, value=30)

i_name_pattern = pn.widgets.Select(name="Name pattern",options=["(?P<rep>.*)_(?P<frac>.*)", ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)",".* (?P<rep>.*)_(?P<frac>.*)",
                                                                ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)", ".* (?P<cond>.*)_(?P<frac>.*_.*)_(?P<rep>.*)", "Custom"])
i_custom_namepattern = pn.widgets.TextInput(name="Customized Name Pattern", placeholder="Enter a string here...e.g.: .* (?P<rep>.*)_(?P<frac>.*)")
regex_pattern = {
    "(?P<rep>.*)_(?P<frac>.*)" : ["Spectronaut MAP1_03K"],
    ".* (?P<rep>.*)_(?P<frac>.*)" : ["MAP1_03K","MAP3_03K"],
    ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)" : ["EGF_rep1_06K","EGF_rep3_06K"],
    ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)" : ["Control_Mem_1", "Control_Cyt_1"],
    ".* (?P<cond>.*)_(?P<frac>.*_.*)_(?P<rep>.*)" : ["4h_mem_frac3_1", "co_mem_frac2_2"]
    }
i_pattern_examples = pn.widgets.Select(name = "Examples", options=regex_pattern[i_name_pattern.value])

@pn.depends(i_name_pattern.param.value, i_custom_namepattern, i_pattern_examples)
def custimization(name_pattern, custom_namepattern, pattern_examples):
    if name_pattern == "Custom":
        return i_custom_namepattern
    else:
        example_for_name_pattern = regex_pattern[name_pattern]
        i_pattern_examples.options = example_for_name_pattern
        return i_pattern_examples

i_x_vs_yAxis_PCA = {
    "PC1" : ["PC3", "PC2"],
    "PC2" : ["PC1", "PC3"],
    "PC3" : ["PC1", "PC2"],
    }

i_xAxis_PCA = pn.widgets.Select(name="X-Axis", options=["PC1", "PC2","PC3"])
i_yAxis_PCA = pn.widgets.Select(name="Y-Axis", options=i_x_vs_yAxis_PCA[i_xAxis_PCA.value])

i_xAxis_PCA_comp = pn.widgets.Select(name="X-Axis", options=["PC1", "PC2","PC3"])
i_yAxis_PCA_comp = pn.widgets.Select(name="Y-Axis", options=i_x_vs_yAxis_PCA[i_xAxis_PCA_comp.value])

@pn.depends(i_xAxis_PCA_comp.param.value, watch=True)
def custimization_PCA_comp(xAxis_PCA_comp):
    yAxis_PCA_comp = i_x_vs_yAxis_PCA[xAxis_PCA_comp]
    i_yAxis_PCA_comp.options = yAxis_PCA_comp
    #return i_yAxis_PCA_comp

@pn.depends(i_xAxis_PCA.param.value, watch=True)
def custimization_PCA(xAxis_PCA):
    yAxis_PCA = i_x_vs_yAxis_PCA[xAxis_PCA]
    i_yAxis_PCA.options = yAxis_PCA
    #return i_yAxis_PCA

@pn.depends(i_acquisition.param.value, i_consecutiveLFQi, i_summed_MSMS_counts)
def acquisition_response(acquisition, consecutiveLFQi, summed_MSMS_counts):
    if acquisition == "SILAC - MQ":
        return pn.Column(pn.Row(i_name_pattern, custimization), pn.Row(pn.Pane("Stringency filtering")), pn.Row(i_RatioHLcount, i_RatioVariability))
    else:
        return pn.Column(pn.Row(pn.Row(i_name_pattern, custimization)), pn.Row(pn.Pane("Stringency filtering")), pn.Row(i_consecutiveLFQi, i_summed_MSMS_counts))

#define widgets that should be disbled after run==True
wdgts = [i_acquisition,i_name_pattern,i_expname, i_pattern_examples, button_analysis, i_expname, i_organism, i_consecutiveLFQi, i_summed_MSMS_counts, 
         i_RatioHLcount, i_RatioVariability, i_comment]            

@pn.depends(i_file.param.value)
def read_file(file):
    if file is None:
        filereading_status = "No file is uploaded"
        cache_uploaded.value = False
        return filereading_status
    else:
        cache_uploaded.value = False
        try:
            if i_file.filename[-3:] == "xls" or i_file.filename[-3:] == "txt":
                df_original = pd.read_csv(BytesIO(file), sep="\t", comment="#", nrows=5, usecols=lambda x: bool(re.match(SpatialDataSet.regex["imported_columns"], x)), low_memory=True)
            elif i_file.filename[-3:] == "csv":
                df_original = pd.read_csv(BytesIO(file), sep=",", comment="#", nrows=5, usecols=lambda x: bool(re.match(SpatialDataSet.regex["imported_columns"], x)), low_memory=True)
            else:
                return pn.Column("Upload either csv, xls, or txt formatted files.")
            cache_uploaded.value = True
            for wdgt in wdgts:
                wdgt.disabled = False

            return pn.Column(pn.Row(pn.widgets.DataFrame(df_original, height=200, width=600, disabled=True)),#i_class.
                             pn.Row(i_expname), 
                             pn.Row(i_organism, i_acquisition), 
                             pn.Row(acquisition_response), 
                             pn.Row(i_comment),
                             pn.Row(button_analysis))

        except Exception: 
            filereading_status = traceback.format_exc()
            cache_uploaded.value = False
            return filereading_status   
        
        
def execution(event):
    if cache_uploaded.value == False:
        analysis_status.object = "Please upload a file first"
    elif i_expname.value == "":
        analysis_status.object = "Please enter an experiment name first"
    else:        
        dashboard_analysis.objects = []
        cache_run.value = False
        for wdgt in wdgts:
            wdgt.disabled = True
        #if you did already your comparison, but add another experiment afterwards - without reloading your AnylsedDatasets.json
        #for wdgt in wdgts_comparison:
        #    wdgt.disabled = True
        try:
            
            dashboard_analysis.append(i_clusterwidget)
            dashboard_analysis.append(i_mapwidget)
            dashboard_analysis.append(analysis_tabs)
            loading_status.objects = [loading]
            analysis_status.object = "Analysis in progress"
            if i_name_pattern.value == "Custom":
                namePattern = i_custom_namepattern.value
            else:
                namePattern = i_name_pattern.value
            global i_class
            if i_acquisition.value == "SILAC":
                i_class = SpatialDataSet(i_file.filename, i_expname.value, i_acquisition.value, comment=i_comment.value, name_pattern=namePattern, organism=i_organism.value, 
                                         RatioHLcount=i_RatioHLcount.value, RatioVariability=i_RatioVariability.value)
            else:
                i_class = SpatialDataSet(i_file.filename, i_expname.value, i_acquisition.value, comment=i_comment.value, name_pattern=namePattern, organism=i_organism.value,
                                         consecutiveLFQi=i_consecutiveLFQi.value, summed_MSMS_counts=i_summed_MSMS_counts.value)
            analysis_status.object = "Data Reading"
            i_class.data_reading(content=BytesIO(i_file.value))
            analysis_status.object = "Data Processing"
            i_class.processingdf()
            update_object_selector(i_mapwidget, i_clusterwidget)
            i_class.quantity_profiles_proteinGroups()
            analysis_status.object = "PCA"
            i_class.perform_pca()
            i_class.multiple_iterations()
            analysis_status.object = "Calculating Manhattan Distance"
            i_class.distance_calculation()
            analysis_status.object = "Calculating Dynamic Range"
            i_class.dynamic_range()
            analysis_status.object = "Writing Overview Table"
            i_class.results_overview_table()
            analysis_status.object = "Writing Analysed Dataset Dictionary"
            SpatialDataSetComparison.analysed_datasets_dict.update(i_class.analysed_datasets_dict)
            loading_status.objects = [idle]
            analysis_status.object = "Analysis finished! Please open the 'Analysis' tab!"
            cache_run.value = True
            #exc_info = sys.exc_info()
        except:
            for wdgt in wdgts:
                wdgt.disabled = False
            loading_status.objects = [""]
            #The traceback gives no traceback, so out of that there will be still the output: Analysis in progress, although it is not possible. Out of that i removed the traceback
            analysis_status.object = traceback.format_exc()
            cache_run.value = False

button_analysis.on_click(execution)  

def update_object_selector(i_mapwidget, i_clusterwidget):
    i_mapwidget.options = list(i_class.map_names)
    i_clusterwidget.options = list(i_class.markerproteins.keys())
        
               
  
@pn.depends(i_mapwidget.param.value, cache_run.param.value, i_collapse_maps_PCA.param.value, i_clusterwidget.param.value, i_xAxis_PCA.param.value, i_yAxis_PCA.param.value)
def update_data_overview(mapwidget, run, collapse_maps_PCA, clusterwidget, xAxis_PCA, yAxis_PCA):
    try:
        if run == True:
            pca_plot = i_class.global_pca_plot(map_of_interest=mapwidget , cluster_of_interest=clusterwidget, x_PCA=xAxis_PCA, y_PCA=yAxis_PCA, collapse_maps=collapse_maps_PCA)
            log_histogram = i_class.plot_log_data()
            visualization_map = pn.Column(
                    pn.Row(i_collapse_maps_PCA),
                    pn.Row(pn.Pane(pca_plot, width=1000)),
                    pn.Row(i_xAxis_PCA, i_yAxis_PCA),
                    pn.Row(pn.Pane(log_histogram, width=1000))
                    )
            app_tabs.active = 1
            return visualization_map
        else:
            visualization_map = "Run analysis first!"
            return visualization_map
    except Exception:
        update_status = traceback.format_exc()
        return update_status
            

@pn.depends(i_clusterwidget.param.value,i_mapwidget.param.value, cache_run.param.value)
def update_cluster_overview(clusterwidget, mapwidget, run):
    try:
        if run == True:
            list_genes = [goi for goi in i_class.genenames_sortedout_list if goi in i_class.markerproteins[clusterwidget]]
            i_class.cache_cluster_quantified = True
            distance_boxplot = i_class.distance_boxplot(cluster_of_interest=clusterwidget)
            if i_class.cache_cluster_quantified == False:
                return "This protein cluster was not quantified"
            
            else:
                df_quantification_overview = i_class.quantification_overview(cluster_of_interest=clusterwidget)
                profiles_plot = i_class.profiles_plot(map_of_interest = mapwidget, cluster_of_interest=clusterwidget)
                pca_plot = i_class.pca_plot(cluster_of_interest=clusterwidget)
                cluster_overview = pn.Column(
                        pn.Row(pn.Pane(pca_plot, width=500),
                               pn.Pane(distance_boxplot, width=500),
                               pn.Pane(profiles_plot, width=500)),
                        pn.Row(pn.Pane("In total {} proteins across all maps were quantified, whereas the following proteins were not consistently quantified throughout all maps: {}".format(
                                i_class.proteins_quantified_across_all_maps, ", ".join(list_genes)) if len(list_genes) != 0 else
                            "All genes from this cluster are quantified in all maps."), width=1000),
                        pn.Row(pn.widgets.DataFrame(df_quantification_overview, height=200, width=500, disabled=True))  
                        )
                return cluster_overview
        
        else:
            cluster_overview = "Run analysis first!"
            return cluster_overview
    except Exception:
        update_status = traceback.format_exc()
        return update_status
    
    
@pn.depends(i_clusterwidget.param.value, cache_run.param.value)
def update_cluster_details(clusterwidget, run):
    try:
        if run == True:
            cluster_details = i_class.distance_to_median_boxplot(cluster_of_interest = clusterwidget)
            return pn.Row(cluster_details, width=1000)
        else:
            cluster_details = "Run analysis first!"
            return pn.Row(cluster_details, width=1000)
    except Exception:
        update_status = traceback.format_exc()
        return update_status
    
@pn.depends(cache_run.param.value)
def update_quantity(run):
    try:
        if run == True:
            fig_npg, fig_npr, fig_npr_dc, fig_npg_F, fig_npgf_F, fig_npg_F_dc = i_class.plot_quantity_profiles_proteinGroups()
            return pn.Column(
                    pn.Row(pn.Column(fig_npg), pn.Column(fig_npr), pn.Column(fig_npr_dc)) ,
                pn.Row(pn.Column(fig_npg_F), pn.Column(fig_npgf_F), pn.Column(fig_npg_F_dc)))
        else:
            return "Run analysis first!"
    except Exception:
        update_status = traceback.format_exc()
        return update_status
    
    
@pn.depends(cache_run.param.value)
def update_dynamic_range(run):
    try:
        if run == True:
            fig_dynamic_range = i_class.plot_dynamic_range()
            return pn.Row(fig_dynamic_range)
        else:
            return "Run analysis first!"
    except Exception:
        update_status = traceback.format_exc()
        return update_status
            
    

@pn.depends(cache_run.param.value)
def show_tabular_overview(run):
    try:
        if run == True:
            content = pn.Column(
                pn.widgets.DataFrame(pd.read_json(i_class.analysed_datasets_dict[i_expname.value]["Overview table"]), height=200, width=600, disabled=True),
                pn.widgets.FileDownload(callback=table_download, filename="cluster_distances.csv"),
                i_logOR01_selection,
                df01_download_widget,
                pn.widgets.FileDownload(callback=json_download, filename="AnalysedDatasets.json")
            )
            return pn.Row(content, width=1000)
        else:
            content = "Please, upload a file first and press ‘Analyse clusters’"
            return pn.Row(content, width=1000)
    except Exception:
        content = traceback.format_exc()
        return pn.Row(content, width=1000)

@pn.depends(cache_run.param.value)
def table_download(run):
    df = i_class.results_overview_table()
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    return sio


@pn.depends(cache_run.param.value)
def json_download(run):
    sio = StringIO()
    json.dump(
        SpatialDataSetComparison.analysed_datasets_dict, 
        sio, 
        indent=4, 
        sort_keys=True
    )
    sio.seek(0)
    return sio


@pn.depends(cache_run.param.value, i_logOR01_selection.param.value)
def df01_download_widget(run, logOR01_selection):
    if logOR01_selection == "0/1 normalized data":
        return pn.Column(pn.widgets.FileDownload(callback=df01_download, filename = "01_normalized_data.csv"), width=650) 
    if logOR01_selection == "log transformed data":
        return pn.Column(pn.widgets.FileDownload(callback=dflog_download, filename = "log_transformed_data.csv"), width=650)
    else:
        return pn.Column(pn.widgets.FileDownload(callback=df_filteredRawData_download, filename = "stringency_filtered_raw_data.csv"), width=650)

    
@pn.depends(cache_run.param.value)
def df01_download(run):
    df_01 = i_class.reframe_df_01ORlog_for_Perseus(i_class.df_01_stacked)
    sio = StringIO()
    df_01.to_csv(sio)
    sio.seek(0)
    return sio 
    
    
@pn.depends(cache_run.param.value)
def dflog_download(run):
    df_log = i_class.reframe_df_01ORlog_for_Perseus(i_class.df_log_stacked)
    sio = StringIO()
    df_log.to_csv(sio)
    sio.seek(0)
    return sio 


@pn.depends(cache_run.param.value)
def df_filteredRawData_download(run):
    df = i_class.reframe_df_01ORlog_for_Perseus(i_class.df_stringencyFiltered)
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    return sio


loading_status_comparison = pn.Row()
idle_comparison = pn.indicators.LoadingSpinner(value=False, width=100, height=100, color="primary")
loading_comparison = pn.indicators.LoadingSpinner(value=True, width=100, height=100, color="primary")

cache_uploaded_json = pn.widgets.Checkbox(value=False)
cache_run_json = pn.widgets.Checkbox(value=False)
button_comparison = pn.widgets.Button(name="Compare experiments", width=50)
i_jsonFile = pn.widgets.FileInput(name="Upload JSON file for comparison")
i_organism_comparison = pn.widgets.Select(options=list(SpatialDataSet.markerproteins_set.keys()), name="Organism", width=300) #
#i_clusterwidget_comparison = pn.widgets.Select(options=list(SpatialDataSet.markerproteins.keys()), name="Cluster of interest", width=300)
i_clusterwidget_comparison = pn.widgets.Select(options=["Proteasome", "Lysosome"], name="Cluster of interest", width=300)
i_clusters_for_ranking = pn.widgets.CrossSelector(name="Select clusters to be considered for ranking calculation", #value=list(SpatialDataSet.markerproteins.keys()), 
                                                options=["Proteasome", "Lysosome"], size=8)
i_multi_choice = pn.widgets.CrossSelector(name="Select experiments for comparison", value=["a", "b"], options=["a", "b", "c"])
i_ref_exp = pn.widgets.Select(name="Select experiments as reference", options=["a", "b", "c"])
i_scatter_metric = pn.widgets.Select(name="Distance metric",
                                     options=["euclidean distance", "manhattan distance",
                                              "1 - cosine correlation", "1 - pearson correlation"])
i_scatter_consolidation = pn.widgets.Select(name="Consolidation of replicate distances",
                                            options=["median","average","sum"])

dashboard_json = pn.Column("Please, upload a file first and press 'Compare clusters'", name="Comparison", css_classes=["content-width"])
comparison_status = pn.Pane("No datasets were compared yet")
i_collapse_maps = pn.widgets.Checkbox(value=False, name="Collapse maps")
i_collapse_cluster = pn.widgets.Checkbox(value=True, name="Collapse cluster")
i_markerset_or_cluster = pn.widgets.Checkbox(value=False, name="Display only protein clusters")
#i_ranking_boxPlot = pn.widgets.Checkbox(value=False, name="Display box plot")
i_ranking_boxPlot = pn.widgets.RadioBoxGroup(name="Types of ranking", options=["Box plot", "Bar plot - median", "Bar plot - sum"], inline=True)
#i_toggle_sumORmedian = pn.widgets.Toggle(name="Sum or Median", button_type="primary")
i_ExpOverview = pn.Row(pn.Pane("", width=1000))
i_include_dataset = pn.widgets.Checkbox(value=False, name="Include data analysed under 'Analysis' tab")
wdgts_comparison = [button_comparison,i_organism_comparison, i_include_dataset] 
json_dict = {}



@pn.depends(cache_run.param.value, i_jsonFile.param.value)#cache_run.param.value
def open_jsonFile(run, jsonFile):#run
    cache_run_json.value = False
    if run == False and jsonFile is None:
        filereading_status_json = "No file is uploaded"
        cache_uploaded_json.value = False
        return filereading_status_json
    else:
        try:
            cache_uploaded_json.value = False
            if run == True:
                global json_dict
                json_dict.update(SpatialDataSet.analysed_datasets_dict)
            elif jsonFile is not None:
                if i_include_dataset.value == False:
                    json_dict = ast.literal_eval(jsonFile.decode("UTF-8"))
                else:
                    #global json_dict
                    json_dict.update(ast.literal_eval(jsonFile.decode("UTF-8"))) #i_class.
            try:
                dashboard_comparison.objects = dashboard_comparison.objects[0:4]
            except:
                pass
            if hasattr(json_dict, "keys") == False: #i_class.
                return "Your json-File does not fulfill the requirements"
            else:
                i_multi_choice.options = []
                filereading_status_json = list(json_dict.keys())# list(set(list(SpatialDataSet.analysed_datasets_dict.keys()) + )) #i_class.
                cache_uploaded_json.value = True
                for wdgt in wdgts_comparison:
                    wdgt.disabled = False
                return pn.Column(pn.Row("You will compare following experiments:\n {}, and {}".format(", ".join(filereading_status_json[:-1]), filereading_status_json[-1])),
                                 pn.Row(i_include_dataset),
                                 pn.Row(i_organism_comparison),
                                 pn.Row(button_comparison),
                                 )
        
        except Exception: 
            filereading_status_json = traceback.format_exc()
            cache_uploaded_json.value = False
            return filereading_status_json                 

@pn.depends(i_multi_choice.param.value, i_clusterwidget_comparison.param.value, cache_run_json.param.value, i_xAxis_PCA_comp.param.value, i_yAxis_PCA_comp.param.value, 
            i_markerset_or_cluster.param.value)
def update_visualization_map_comparison(multi_choice, clusterwidget_comparison, run_json, xAxis_PCA_comp, yAxis_PCA_comp, markerset_or_cluster):
    try:
        if run_json == True:
            if multi_choice == []:
                return pn.Column(#pn.Row(i_multi_choice),
                                 pn.Row("Please select experiments for comparison")
                                )
            else:
                pass
            pca_global_comparison = i_class_comp.plot_global_pca_comparison(cluster_of_interest_comparison=clusterwidget_comparison, x_PCA=xAxis_PCA_comp, y_PCA=yAxis_PCA_comp, 
                                                                       markerset_or_cluster=markerset_or_cluster, multi_choice=multi_choice)
            if markerset_or_cluster == False:
                return pn.Column(#pn.Row(i_multi_choice),
                                 pn.Row(i_clusterwidget_comparison),
                                 pn.Row(i_markerset_or_cluster),
                                 pn.Row(pca_global_comparison),
                                 pn.Row(i_xAxis_PCA_comp, i_yAxis_PCA_comp)  
                                )
            else:
                return pn.Column(#pn.Row(i_multi_choice),
                                 pn.Row(i_markerset_or_cluster),
                                 pn.Row(pca_global_comparison),
                                 pn.Row(i_xAxis_PCA_comp, i_yAxis_PCA_comp)   
                                )
        else:
            pca_global_comparison = "Run analysis first!"
            return pca_global_comparison
    except Exception:
        update_status = traceback.format_exc()
        return update_status


@pn.depends(i_multi_choice.param.value, i_clusterwidget_comparison.param.value, i_collapse_maps.param.value, i_collapse_cluster.param.value, i_clusters_for_ranking.param.value, i_ref_exp.param.value,
            i_ranking_boxPlot.param.value, cache_run_json.param.value) #i_toggle_sumORmedian.param.value, toggle_sumORmedian
def update_distance_and_pca(multi_choice, clusterwidget_comparison, collapse_maps, collapse_cluster, clusters_for_ranking, ref_exp, ranking_boxPlot, run_json):
    try:
        if run_json == True:
            if multi_choice == []:
                return pn.Column(#pn.Row(i_multi_choice),
                                 pn.Row("Please select experiments for comparison")
                                )
            else:
                pass
            #i_ref_exp.options = multi_choice
            update_ref_exp(i_ref_exp)
            if clusters_for_ranking == []:
                clusters_for_ranking = [clusterwidget_comparison]
            else:
                pass
            
            distance_ranking_comparison = i_class_comp.distance_ranking_barplot_comparison(collapse_cluster=collapse_cluster, multi_choice=multi_choice, 
                                                                                           clusters_for_ranking=clusters_for_ranking, ranking_boxPlot=ranking_boxPlot)
            i_class_comp.cache_cluster_quantified = True
            distance_comparison = i_class_comp.distance_boxplot_comparison(collapse_maps=collapse_maps, cluster_of_interest_comparison=clusterwidget_comparison, multi_choice=multi_choice)
            if i_class_comp.cache_cluster_quantified == False:
                return pn.Column(
                                 pn.Row(i_clusterwidget_comparison),
                                 pn.Row("Cluster was not quantified in any experiment")
                                )
            else:
                pca_comparison = i_class_comp.plot_pca_comparison(collapse_maps=collapse_maps, cluster_of_interest_comparison=clusterwidget_comparison, multi_choice=multi_choice)
                if clusters_for_ranking == []:
                    return pn.Column(
                                     pn.Row(i_clusters_for_ranking),
                                     pn.Row("Select at least one cluster"),         
                                     pn.Row(i_clusterwidget_comparison),
                                     pn.Row(i_collapse_maps),
                                     pn.Row(pca_comparison),
                                     pn.Row(distance_comparison),
                                     )
                else:
                    if i_collapse_cluster.value == True:
                        return pn.Column(
                                     pn.Row(i_clusters_for_ranking),
                                     pn.Row(i_collapse_cluster, i_ranking_boxPlot),
                                     pn.Row(distance_ranking_comparison),
                                     #pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster, height=200, width=1800, disabled=True)),
                                     pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster2, height=200, width=1800, disabled=True)),
                                     pn.Row(i_clusterwidget_comparison),
                                     pn.Row(i_collapse_maps),
                                     pn.Row(pca_comparison),
                                     pn.Row(distance_comparison),

                                    )
                    else:
                        return pn.Column(
                                     pn.Row(i_clusters_for_ranking),
                                     pn.Row(i_collapse_cluster),
                                     pn.Row(distance_ranking_comparison),
                                     #pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster, height=200, width=1800, disabled=True)), 
                                     pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster2, height=200, width=1800, disabled=True)),
                                     pn.Row(i_clusterwidget_comparison),
                                     pn.Row(i_collapse_maps),
                                     pn.Row(pca_comparison),
                                     pn.Row(distance_comparison),

                                    )
        else:
            pca_comparison = "Run analysis first!"
            return pca_comparison
    except Exception:
        update_status = traceback.format_exc()
        return update_status
    
    
@pn.depends(i_multi_choice.param.value, cache_run_json.param.value)
def update_npr_ngg_nprDc(multi_choice, run_json):
    try:
        if run_json == True: 
            if multi_choice == []:
                return pn.Column(
                                 pn.Row("Please select experiments for comparison"))
            else:
                fig_quantity_pg, fig_quantity_pr = i_class_comp.quantity_pr_pg_barplot_comparison(multi_choice=multi_choice)
                coverage_barplot = i_class_comp.coverage_comparison(multi_choice=multi_choice)
                return pn.Column(
                                 pn.Row(fig_quantity_pg), 
                                 pn.Row(fig_quantity_pr),
                                 pn.Row(coverage_barplot)
                                )
        else:
            completeness_barplot = "Run analysis first!"
            return completeness_barplot
    except Exception:
        update_status = traceback.format_exc()
        return update_status 
    
@pn.depends(i_multi_choice.param.value, cache_run_json.param.value)
def update_venn(multi_choice, run_json):
    try:
        if run_json == True: 
            venn_plot = []
            if len(multi_choice)<=1:
                return pn.Column(
                                 pn.Row(pn.Pane("Please select 2 or more experiments for comparison"), width=1000))
            else:
                venn_plot, figure_UpSetPlot = i_class_comp.venn_sections(multi_choice_venn = multi_choice)
                return pn.Column(
                                 pn.Pane(venn_plot),
                                 pn.Row(figure_UpSetPlot,width=1000)
                                )
        else:
            venn_plot = "Run analysis first!"
            return venn_plot
    except Exception:
        update_status = traceback.format_exc()
        return update_status    

@pn.depends(i_multi_choice.param.value, cache_run_json.param.value)
def update_SVM_Analysis(multi_choice, run_json):
    try:
        if run_json == True: 
            if multi_choice == []:
                return pn.Column(
                                 pn.Row("Please select experiments for comparison"))
            else:
                if i_class_comp.cache_stored_SVM == False:
                    return pn.Column(
                                 pn.Row("No Missclassifiaction Matrix is stored"))
                else:
                    fig_markerPredictionAccuracy, fig_clusterPerformance = i_class_comp.svm_plotting(multi_choice)
                    return pn.Column(
                                 pn.Row(fig_markerPredictionAccuracy),
                                 pn.Row(fig_clusterPerformance))
    except Exception:
        update_status = traceback.format_exc()
        return update_status 
                
@pn.depends(i_multi_choice.param.value, i_ref_exp.param.value, i_collapse_cluster.param.value, cache_run_json.param.value)
def update_dynamic_range_comparison(multi_choice, ref_exp, collapse_cluster, run_json):
    try:
        if run_json == True: 
            if multi_choice == []:
                return pn.Column(
                                 pn.Row("Please select experiments for comparison"))
            else:
                dynamic_range_barplot = i_class_comp.dynamic_range_comparison(collapse_cluster=collapse_cluster, multi_choice=multi_choice, ref_exp=ref_exp)
                return pn.Column(
                                 pn.Row(dynamic_range_barplot),
                                 pn.Row(i_collapse_cluster),
                                 pn.Row(i_ref_exp)
                                )
        else:
            dynamic_range_barplot = "Run analysis first!"
            return dynamic_range_barplot
    except Exception:
        update_status = traceback.format_exc()
        return update_status 
    
    
@pn.depends(i_multi_choice.param.value, i_scatter_metric.param.value, i_scatter_consolidation.param.value, cache_run_json.param.value)
def update_global_scatter_comparison(multi_choice, metric, consolidation, run_json):
    try:
        if run_json == True: 
            if multi_choice == []:
                return pn.Column(#pn.Row(i_multi_choice),
                                 pn.Row("Please select experiments for comparison"))
            else:
                scatter_histogram = i_class_comp.calculate_global_scatter(multi_choice=multi_choice,
                                                                              metric=metric, consolidation=consolidation)
                return pn.Column(pn.Row(i_scatter_metric),
                                 pn.Row(i_scatter_consolidation),
                                 pn.Row(scatter_histogram)
                                )
        else:
            return "Run analysis first!"
    except Exception:
        update_status = traceback.format_exc()
        return update_status
    
    
def update_multi_choice(i_multi_choice, i_clusterwidget, i_clusters_for_ranking):
    i_multi_choice.options = list(json_dict.keys())
    i_clusterwidget.options = list(i_class_comp.markerproteins.keys())
    i_clusters_for_ranking.options = list(i_class_comp.markerproteins.keys())
    i_clusters_for_ranking.value = list(i_class_comp.markerproteins.keys())
    i_multi_choice.value = list(json_dict.keys())

    
@pn.depends(i_multi_choice.param.value, watch=True)
def update_ref_exp(multi_choice):
    i_ref_exp.options = i_multi_choice.value
    #return i_ref_exp


@pn.depends(i_multi_choice.param.value, watch=True)
def update_ExpOverview(multi_choice):
    dict_analysis_parameters={}
    for exp_name in multi_choice:
        dict_analysis_parameters[exp_name] = json_dict[exp_name]["Analysis parameters"]
    i_ExpOverview[0] = pn.widgets.DataFrame(pd.DataFrame.from_dict(dict_analysis_parameters))
    i_ExpOverview.value = pd.DataFrame.from_dict(dict_analysis_parameters)
    i_ExpOverview.disabled = True
    i_ExpOverview.height = 200

    
def execution_comparison(event):
    if cache_uploaded_json.value == False:
        comparison_status.object = "Please upload a JSON-file first"
    else:        
        #dashboard_comparison.objects[2:] = []
        cache_run_json.value = False
        for wdgt in wdgts_comparison:
            wdgt.disabled = True
        try:
            loading_status_comparison.objects = [loading_comparison]
            comparison_status.object = "Analysis in progress"
            protein_cluster = SpatialDataSet.markerproteins_set[i_organism_comparison.value].keys()
            update_ref_exp(i_ref_exp)
            global i_class_comp
            comparison_status.object = "Initialization"
            i_class_comp = SpatialDataSetComparison(clusters_for_ranking=protein_cluster, ref_exp=i_ref_exp.value, organism=i_organism_comparison.value)
            i_class_comp.json_dict = json_dict
            comparison_status.object = "Reading"
            i_class_comp.read_jsonFile()
            update_multi_choice(i_multi_choice, i_clusterwidget_comparison, i_clusters_for_ranking)
            comparison_status.object = "SVM Processing"
            i_class_comp.svm_processing()
            comparison_status.object = "PCA"
            i_class_comp.perform_pca_comparison()
            dashboard_comparison.append(pn.Row(i_multi_choice))
            dashboard_comparison.append(pn.Row(i_ExpOverview))
            dashboard_comparison.append(comparison_tabs)
            loading_status_comparison.objects = [idle_comparison]
            comparison_status.object = "Comparison finished!"
            cache_run_json.value = True
        except Exception:
            loading_status_comparison.objects = [""]
            for wdgt in wdgts_comparison:
                wdgt.disabled = False
            comparison_status.object = traceback.format_exc()
            cache_run_json.value = False
button_comparison.on_click(execution_comparison)


i_jsonFile_amendments_intended = pn.widgets.FileInput(name="Upload JSON file to be amended")
i_json_ExpSelector = pn.widgets.CrossSelector(name="Select experiments, that will be removed from JSON file", width=1000)
cache_uploaded_json_amendment = pn.widgets.Checkbox(value=False)
cache_run_json_amendment = pn.widgets.Checkbox(value=False)
button_reset = pn.widgets.Button(name="Reset", width=650)
download_status = pn.Pane("Upload a JSON file first", width=1000)
i_df_ExpComment = pn.widgets.DataFrame()
wdgt_json = [button_reset]
json_dict_amendments_intended = {}
#make a cache, and say, if this hasnt been executed so far, please reset it
dict_new_expNames = {}
dict_new_comments = {}
i_exp_SVM = pn.widgets.Select(name="Select experiments as reference", options=["a", "b", "c"])
button_SVM_analysis = pn.widgets.Button(name="Analyse misclassification matrix", width=50)
i_SVM_table = pn.widgets.input.TextAreaInput(name="Misclassification matrix from Perseus", placeholder="Copy matrix here...")
cache_uploaded_SVM = pn.widgets.Checkbox(value=False)
analysis_status_SVM = pn.Row(pn.Pane("No SVM analysis run yet", width=1000))
i_all_or_marker = pn.widgets.Select(name="Select dataset for Perseus", options=["0/1 normalized data, all experiments", 
                                                                               "0/1 normalized data, markerset only, all experiments"])
    
@pn.depends(i_jsonFile_amendments_intended.param.value)#cache_run.param.value
def open_jsonFile_amendment(jsonFile_amendments):#run
    cache_run_json_amendment.value = False
    if jsonFile_amendments is None:
        cache_uploaded_json_amendment.value = False
    else:
        dashboard_manageDatasets.objects[2:] = []
        #dashboard_manageDatasets.objects = []
        dashboard_MissclassificationMatrix.objects = []
        dashboard_amendment.objects = []
        cache_uploaded_json_amendment.value = False
        try:
            
            json_dict_cache = ast.literal_eval(i_jsonFile_amendments_intended.value.decode("UTF-8"))
            if hasattr(json_dict_cache, "keys") == False: 
                    return "Your json-File does not fulfill the requirements"
            else:
                global json_dict_amendments_intended
                try:
                    json_dict_amendments_intended.update(json_dict_cache)
                except Exception:
                    json_dict_amendments_intended = json_dict_cache
                i_json_ExpSelector.options = list(json_dict_amendments_intended.keys())
                cache_uploaded_json_amendment.value = True
                for wdgt in wdgt_json:
                    wdgt.disabled = False
                download_status.object = "Upload successful! Select experiments now."
                dashboard_manageDatasets.append(amendment_tabs)
                dashboard_amendment.append(pn.Column(i_df_ExpComment, download_status))
                analysis_status_SVM[0] = pn.Pane("Upload successful! Select experiments now.")
                dashboard_MissclassificationMatrix.append(analysis_status_SVM)
                return pn.Column(
                                 i_json_ExpSelector,
                                 pn.widgets.FileDownload(callback=json_amendment_download, filename="AnalysedDatasets2.0.json", width=650),
                                 i_all_or_marker,           
                                 df01_json_download_widget,#pn.Column(pn.widgets.FileDownload(callback=df01_fromJson_download, filename = "all_01_normalized_data.csv", width=650)),
                                 button_reset,
                                 )
        except Exception: 
            filereading_status_json = traceback.format_exc()
            cache_uploaded_json_amendment.value = False
            return filereading_status_json

    
@pn.depends(i_exp_SVM.param.value, watch=True)
def update_SVM_Matrix(exp_SVM):
    try:
        i_SVM_table.value = json_dict_amendments_intended[exp_SVM]["Misclassification Matrix"]
    except:
        i_SVM_table.value = ''
    

@pn.depends(i_json_ExpSelector.param.value, watch=True)
def update_exp_for_SVM(json_ExpSelector):
    if json_ExpSelector == []:
        dashboard_MissclassificationMatrix.objects = []
        analysis_status_SVM[0] = pn.Pane("Select experiments first")
        dashboard_MissclassificationMatrix.append(analysis_status_SVM)
    else:
        #i_SVM_table.value = ""       
        dashboard_MissclassificationMatrix.objects = []
        i_exp_SVM.options = json_ExpSelector
        analysis_status_SVM[0] = pn.Pane("Please paste a SVM Matrix first")
        dashboard_MissclassificationMatrix.append(pn.Row(i_exp_SVM, i_SVM_table))
        dashboard_MissclassificationMatrix.append(read_SVM_matrix)
        dashboard_MissclassificationMatrix.append(analysis_status_SVM)
    
        
@pn.depends(i_SVM_table.param.value)
def read_SVM_matrix(SVM_table):   
    if SVM_table == "":
        SVM_reading_status = "No misclassification matrix is uploaded"
        cache_uploaded_SVM.value = False
        analysis_status_SVM[0] = pn.Pane("Please paste a SVM Matrix first")
    else:
        cache_uploaded_SVM.value = False
        try:
            try:
                df_SVM = pd.read_json(json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"])
            except KeyError:
                df_SVM = pd.read_table(StringIO(SVM_table), sep="\t")
                json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"] = df_SVM.to_json()
            #SVM_dict, SVM_dict_total = i_class.svm_processing()
            
                #"Cluster performance" : SVM_dict.copy(), 
            #                                                                                "Total cluster performance" : SVM_dict_total.copy()}
            ##analysis_status_SVM[0] = pn.Pane("No SVM analysis run yet")
            SVM_reading_status = svm_heatmap(df_SVM)
            cache_uploaded_SVM.value = True
            #button_SVM_analysis.disabled = False
            return pn.Column(#pn.Row(button_SVM_analysis),
                             pn.Row(SVM_reading_status, height=600),
                            )
        except Exception: 
            SVM_reading_status = "Paste the SVM matrix from Perseus only!"
            #SVM_reading_status = traceback.format_exc()
            cache_uploaded_SVM.value = False
            analysis_status_SVM[0] = pn.Pane(traceback.format_exc())    #pn.Pane("")
            return SVM_reading_status 


@pn.depends(cache_run_json_amendment.param.value)
def df01_fromJson_download(run):
    if i_json_ExpSelector.value == []:
        download_status.object = "No experiments are selected"
        return
    else:
        df = reframe_df_01_fromJson_for_Perseus(json_dict_amendments_intended)
        sio = StringIO()
        df.to_csv(sio)
        sio.seek(0)
        return sio 

    
@pn.depends(cache_run_json_amendment.param.value)
def df01_marker_fromJson_download(run):
    if i_json_ExpSelector.value == []:
        download_status.object = "No experiments are selected"
        return
    else:
        df = reframe_df_01_fromJson_for_Perseus(json_dict_amendments_intended)
        df = df.loc[df.index.get_level_values("Compartment")!= "undefined"]
        sio = StringIO()
        df.to_csv(sio)
        sio.seek(0)
        return sio 
    
    
@pn.depends(cache_run_json_amendment.param.value, i_all_or_marker.param.value)
def df01_json_download_widget(run, all_or_marker):
    if all_or_marker == "0/1 normalized data, all experiments":
        return pn.Column(pn.widgets.FileDownload(callback=df01_fromJson_download, filename = "all_01_normalized_data.csv"), width=650) 
    else:
        return pn.Column(pn.widgets.FileDownload(callback=df01_marker_fromJson_download, filename = "marker_01_normalized_data.csv"), width=650)
    
    
def json_amendment_download():
    if i_json_ExpSelector.value == []:
        download_status.object = "No experiments are selected"
        return
    else:
        json_new = json_dict_amendments_intended.copy()
        exp_names_del = [elem for elem in i_json_ExpSelector.options if not elem in i_json_ExpSelector.value]        
        for key in exp_names_del:
            del json_new[key]
        checked_exp = set()
        redundant_expNames = set(new_Exp for new_Exp in dict_new_expNames.values() if new_Exp in checked_exp or checked_exp.add(new_Exp))
        if redundant_expNames != set():
            download_status.object = "Experiments are not allowed to be labelled identically"
            return
        else:
            for exp_name in json_new:
                json_new[exp_name]["Analysis parameters"]["comment"] = dict_new_comments[exp_name] #dict_new_expNames
            json_new = {dict_new_expNames[oldK]: value for oldK, value in json_new.items()}
            sio = StringIO()
            json.dump(
                json_new, 
                sio, 
                indent=4, 
                sort_keys=True
            )
            sio.seek(0)
            download_status.object = "Download sucessful"
            return sio

        
def reset_json_amendment(event):
    global json_dict_amendments_intended
    json_dict_amendments_intended = {}
    i_json_ExpSelector.options = []
    i_json_ExpSelector.value = []
    i_df_ExpComment.value = pd.DataFrame()
    for wdgt in wdgt_json:
        wdgt.disabled = True
    download_status.object = "Reset sucessful"
button_reset.on_click(reset_json_amendment)


@pn.depends(i_json_ExpSelector.param.value, watch=True)
def update_renameExp(json_ExpSelector):
    dict_ExpComments = {}
    for exp_name in json_ExpSelector:
        dict_ExpComments[exp_name] = json_dict_amendments_intended[exp_name]["Analysis parameters"]["comment"]
    df_ExpComments = pd.DataFrame(dict_ExpComments.items(), columns=["Experiment name - old", "Comment"])#pd.DataFrame.from_dict(dict_ExpComments)
    df_ExpComments.insert(0, "Experiment name - new", df_ExpComments["Experiment name - old"])
    df_ExpComments.set_index("Experiment name - old", inplace=True)
    df_ExpComments.replace({"Experiment name - new": dict_new_expNames}, inplace=True)
    exp_previous = list(dict_new_comments.keys())
    for exp in exp_previous:
        if exp not in json_ExpSelector:
            del dict_new_comments[exp]
    df_ExpComments.loc[dict_new_comments.keys(),"Comment"] = list(dict_new_comments.values())
    i_df_ExpComment.value = df_ExpComments
    i_df_ExpComment.height=len(json_ExpSelector)*50
    #return i_df_ExpComment

@pn.depends(i_df_ExpComment.param.value, watch=True)
def update_newExpNames(df_ExpComment):
    try:        
        global dict_new_expNames 
        changed_expName = set(list(dict_new_expNames.values())+list(df_ExpComment["Experiment name - new"]))-set(dict_new_expNames.values())
        dict_new_expNames = dict(zip(df_ExpComment.index, df_ExpComment["Experiment name - new"]))
        global dict_new_comments
        dict_new_comments_cache = dict_new_comments.copy()
        changed_comment = set(list(dict_new_comments.values())+list(df_ExpComment["Comment"]))-set(dict_new_comments.values())
        dict_new_comments = dict(zip(df_ExpComment.index, df_ExpComment["Comment"]))
        if changed_expName == set() and changed_comment == set():
            tracked_change = "No changes saved"
        elif changed_expName != set() and changed_comment != set():
            tracked_change = "Upload sucessful"
        elif changed_expName != set():
            new_exp = list(changed_expName)[0]
            old_exp = list(dict_new_expNames.keys())[list(dict_new_expNames.values()).index(new_exp)]
            tracked_change = "Experiment name was changed from {} to {}".format(old_exp, new_exp)
        else:# changed_comment != set():
            new_comment = list(changed_comment)[0]
            exp = list(dict_new_comments.keys())[list(dict_new_comments.values()).index(new_comment)]
            old_comment = dict_new_comments_cache[exp]
            tracked_change = "Comment of the experiment {} was changed from {} to {}".format(exp, old_comment, new_comment)
        download_status.object = tracked_change
    except Exception:
        pass

dasboard_home = pn.Column(i_file, read_file,analysis_status, loading_status, name="Home", css_classes=["content-width"])
dashboard_analysis = pn.Column("Please, upload a file first and press 'Analyse clusters'", name="Analysis", css_classes=["content-width"])
dashboard_MissclassificationMatrix = pn.Column("Please, upload a file first and press 'Analyse clusters'", name="SVM Analysis", css_classes=["content-width"])
dashboard_amendment = pn.Column("Please, upload a json file first", name="Renaming", css_classes=["content-width"])
dashboard_comparison = pn.Column(i_jsonFile, open_jsonFile, comparison_status, loading_status_comparison)
dashboard_manageDatasets = pn.Column(i_jsonFile_amendments_intended, open_jsonFile_amendment)
    
analysis_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
analysis_tabs.append(("Data overview", update_data_overview))
analysis_tabs.append(("Cluster Overview", update_cluster_overview))
analysis_tabs.append(("Cluster Details", update_cluster_details))
analysis_tabs.append(("Depth and Coverage", update_quantity))
analysis_tabs.append(("Dynamic Range", update_dynamic_range))
analysis_tabs.append(("Download", show_tabular_overview))

comparison_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
comparison_tabs.append(("Data Overview", update_visualization_map_comparison))
comparison_tabs.append(("Global Scatter", update_global_scatter_comparison))
comparison_tabs.append(("Biological Precision", update_distance_and_pca))
comparison_tabs.append(("Depth and Coverage", update_npr_ngg_nprDc))
comparison_tabs.append(("Unique and shared protein groups", update_venn))
comparison_tabs.append(("Dynamic Range", update_dynamic_range_comparison))
comparison_tabs.append(("SVM Analysis", update_SVM_Analysis))

amendment_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
amendment_tabs.append(("Change Experiment name and comment", dashboard_amendment))
amendment_tabs.append(("SVM Upload", dashboard_MissclassificationMatrix))


app_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
app_tabs.append(("Home", dasboard_home))
app_tabs.append(("Analysis", dashboard_analysis))
app_tabs.append(("Data comparison", dashboard_comparison))
app_tabs.append(("Manage Datasets", dashboard_manageDatasets))

app_tabs.append(("About", pn.Row("Explanation of what's going on here", width=2000)))

#i_search = pn.widgets.TextInput(name="Search")
app_center = pn.Column(pn.Row(pn.Pane("# QC tool for Spatial Proteomics", width = 600),
                              pn.layout.HSpacer(),
                              #i_search,
                              #width=1600, 
                              margin=10),
                       app_tabs,
                       #pn.Spacer(background="#DDDDDD", height=100, margin=0)
                      )
app = pn.GridSpec()#sizing_mode="stretch_both", margin=0)
app[0,0] = pn.Spacer(background="white", margin=0) #"#DDDDDD"
app[0,9] = pn.Spacer(background="white", margin=0) #"#DDDDDD"
app[0,1:8] = app_center

pwd = pn.widgets.PasswordInput(name="Please enter password for access.")
app_container = pn.Column(pwd)

def check_pwd(event, app=app):
    pwd = event.new
    if pwd == "pwd":
        app_container[0]=app
pwd.param.watch(check_pwd, "value")

try:
    server.stop()
except Exception:
    print("First server startup")
server = app.show(port=5065, websocket_max_message_size=2000000000)
#app.servable()

Launching server at http://localhost:5065


In [131]:
i_json_ExpSelector.value

[]

In [122]:
for exp_name in json_dict_amendments_intended.keys():
    for data_type in json_dict_amendments_intended[exp_name].keys():                   
        if data_type == "0/1 normalized data" and exp_name == list(json_dict_amendments_intended.keys())[0]:
            df_01_combined = pd.read_json(json_dict_amendments_intended[exp_name][data_type])
            df_01_combined = df_01_combined.set_index(["Gene names", "Protein IDs", "Compartment"]).copy()
            df_01_combined.drop([col for col in df_01_combined.columns if not col.startswith("normalized profile")])
            df_01_combined.columns = pd.MultiIndex.from_tuples([el.split("?") for el in df_01_combined.columns], names=["Set", "Map", "Fraction"])
            df_01_combined.rename(columns = {"normalized profile":exp_name}, inplace=True)

        elif data_type == "0/1 normalized data" and exp_name != list(json_dict_amendments_intended.keys())[0]:
            df_01_toadd = pd.read_json(json_dict_amendments_intended[exp_name][data_type])
            df_01_toadd = df_01_toadd.set_index(["Gene names", "Protein IDs", "Compartment"]).copy()
            df_01_toadd.drop([col for col in df_01_toadd.columns if not col.startswith("normalized profile")])
            df_01_toadd.columns = pd.MultiIndex.from_tuples([el.split("?") for el in df_01_toadd.columns], names=["Set", "Map", "Fraction"])
            df_01_toadd.rename(columns = {"normalized profile":exp_name}, inplace=True)
            df_01_combined = pd.concat([df_01_combined, df_01_toadd], axis=1)#, join="inner")  
                            
#filter for consistently quantified proteins (they have to be in all fractions and all maps)
#df_01_filtered_combined = df_01_mean_combined.dropna()    
df_01_combined.columns.names = ["Experiment", "Map", "Fraction"]
#reframe it to make it ready for PCA
df_01_filtered_combined = df_01_combined.stack(["Experiment", "Map"]).dropna(axis=0)
#df_01_filtered_combined = df_01_combined.stack(["Experiment"]).dropna(axis=1)
df_01_filtered_combined = df_01_filtered_combined.div(df_01_filtered_combined.sum(axis=1), axis=0)
index_ExpMap = df_01_filtered_combined.index.get_level_values("Experiment")+"_"+df_01_filtered_combined.index.get_level_values("Map")
index_ExpMap.name = "Exp_Map"
df_01_filtered_combined.set_index(index_ExpMap, append=True, inplace=True)

df_01_filtered_combined.index = df_01_filtered_combined.index.droplevel(["Map", "Experiment"])
df_01_filtered_combined = df_01_filtered_combined.stack("Fraction").unstack(["Exp_Map", "Fraction"])
df_01_filtered_combined.columns = ["_".join(col) for col in df_01_filtered_combined.columns.values]
df_01_filtered_combined

autophagy_Nt_rep1_01K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.120844   
           O95819-3    undefined                 0.118382   
           P09493-3    undefined                 0.218229   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.078236   
ZYG11B     Q9C0D3      undefined                 0.093879   
ZYX        Q15942      undefined                 0.182766   
ZZEF1      O43149      undefined                 0.127042   
ZZZ3       Q8IYH5      undefined                 0.361936   

                                    autophagy_Nt_rep1_03K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.068663   
           O95819-3    undefined                 0.165104   
           P09493-3    undefined                 0.198378   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.125777   
ZYG11B     Q9C0D3      undefined                 0.228101   
ZYX        Q15942      undefined                 0.227020   
ZZEF1      O43149      undefined                 0.170573   
ZZZ3       Q8IYH5      undefined                 0.117032   

                                    autophagy_Nt_rep1_06K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.123748   
           O95819-3    undefined                 0.173973   
           P09493-3    undefined                 0.137407   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.143690   
ZYG11B     Q9C0D3      undefined                 0.172156   
ZYX        Q15942      undefined                 0.159561   
ZZEF1      O43149      undefined                 0.067812   
ZZZ3       Q8IYH5      undefined                 0.071840   

                                    autophagy_Nt_rep1_12K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.159507   
           O95819-3    undefined                 0.218474   
           P09493-3    undefined                 0.168588   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.225183   
ZYG11B     Q9C0D3      undefined                 0.252722   
ZYX        Q15942      undefined                 0.158012   
ZZEF1      O43149      undefined                 0.213065   
ZZZ3       Q8IYH5      undefined                 0.277874   

                                    autophagy_Nt_rep1_24K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.258687   
           O95819-3    undefined                 0.180889   
           P09493-3    undefined                 0.144000   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.229102   
ZYG11B     Q9C0D3      undefined                 0.191027   
ZYX        Q15942      undefined                 0.192820   
ZZEF1      O43149      undefined                 0.292642   
ZZZ3       Q8IYH5      undefined                 0.074801   

                                    autophagy_Nt_rep1_80K  \
Gene 

In [142]:
df_01_filtered_combined.loc[df_01_filtered_combined.index.get_level_values("Compartment")!= "undefined"]

,,,autophagy_Nt_rep1_01K,autophagy_Nt_rep1_03K,autophagy_Nt_rep1_06K,autophagy_Nt_rep1_12K,autophagy_Nt_rep1_24K,autophagy_Nt_rep1_80K,autophagy_St_rep1_01K,autophagy_St_rep1_03K,autophagy_St_rep1_06K,autophagy_St_rep1_12K,...,r_Map2_06K,r_Map2_12K,r_Map2_24K,r_Map2_80K,r_Map3_01K,r_Map3_03K,r_Map3_06K,r_Map3_12K,r_Map3_24K,r_Map3_80K
Gene names,Protein IDs,Compartment,,,,,,,,,,,,,,,,,,,,,
AARS2,Q5JTZ9,Mitochondrion,0.106289,0.350874,0.297433,0.088276,0.073599,0.083530,0.116513,0.322074,0.280905,0.074271,...,0.336253,0.037157,0.010908,0.009860,0.066749,0.461608,0.426950,0.034183,0.006850,0.003660
ABCB1,P08183,Plasma membrane,0.138130,0.213086,0.263010,0.196984,0.115675,0.073114,0.159619,0.248668,0.240216,0.190264,...,0.328687,0.124952,0.046598,0.028367,0.152404,0.291325,0.298280,0.162765,0.067391,0.027834
ABCC2,Q92887,Plasma membrane,0.146270,0.202098,0.214754,0.252312,0.114053,0.070514,0.133363,0.153984,0.203794,0.260908,...,0.299570,0.185207,0.089369,0.065218,0.164224,0.224272,0.291325,0.184211,0.074442,0.061527
ABCD1,P33897,Peroxisome,0.080507,0.153021,0.309539,0.275523,0.122681,0.058728,0.074568,0.142612,0.296151,0.310737,...,0.392109,0.278047,0.063975,0.009434,0.072491,0.140287,0.409776,0.278424,0.089489,0.009532
ABCD3,P28288,Peroxisome,0.063482,0.141136,0.320021,0.336485,0.116216,0.022659,0.069629,0.141101,0.326087,0.339821,...,0.361129,0.249702,0.094149,0.014942,0.072308,0.180157,0.356030,0.269758,0.109771,0.011977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAG,P61981,Large Protein Complex,0.119828,0.172074,0.155495,0.212093,0.184195,0.156315,0.159977,0.148574,0.142615,0.221994,...,0.202328,0.155304,0.132680,0.153995,0.174460,0.230973,0.195898,0.119813,0.137730,0.141125
YWHAH,Q04917,Large Protein Complex,0.153475,0.172848,0.158939,0.206247,0.160903,0.147589,0.184653,0.157112,0.150878,0.196821,...,0.141613,0.127899,0.223052,0.190712,0.169349,0.176586,0.181380,0.124901,0.173492,0.174292
YWHAQ,P27348,Large Protein Complex,0.157208,0.184107,0.159925,0.196513,0.168570,0.133677,0.160972,0.152548,0.153458,0.229508,...,0.136761,0.161491,0.177852,0.134985,0.148362,0.179313,0.163409,0.165259,0.182426,0.161231


In [141]:
df_01_filtered_combined.index.get_level_values("Compartment").unique()

Index(['undefined', 'Mitochondrion', 'Plasma membrane', 'Peroxisome',
       'Actin binding proteins', 'Lysosome', 'Large Protein Complex', 'ER',
       'Endosome', 'Golgi', 'ER_high_curvature', 'Ergic/cisGolgi',
       'Nuclear pore complex'],
      dtype='object', name='Compartment')

In [137]:
df_01_filtered_combined.loc[df_01_filtered_combined.index.get_level_values("Compartment").isin(multi_choice)].copy()]

autophagy_Nt_rep1_01K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.120844   
           O95819-3    undefined                 0.118382   
           P09493-3    undefined                 0.218229   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.078236   
ZYG11B     Q9C0D3      undefined                 0.093879   
ZYX        Q15942      undefined                 0.182766   
ZZEF1      O43149      undefined                 0.127042   
ZZZ3       Q8IYH5      undefined                 0.361936   

                                    autophagy_Nt_rep1_03K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.068663   
           O95819-3    undefined                 0.165104   
           P09493-3    undefined                 0.198378   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.125777   
ZYG11B     Q9C0D3      undefined                 0.228101   
ZYX        Q15942      undefined                 0.227020   
ZZEF1      O43149      undefined                 0.170573   
ZZZ3       Q8IYH5      undefined                 0.117032   

                                    autophagy_Nt_rep1_06K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.123748   
           O95819-3    undefined                 0.173973   
           P09493-3    undefined                 0.137407   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.143690   
ZYG11B     Q9C0D3      undefined                 0.172156   
ZYX        Q15942      undefined                 0.159561   
ZZEF1      O43149      undefined                 0.067812   
ZZZ3       Q8IYH5      undefined                 0.071840   

                                    autophagy_Nt_rep1_12K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.159507   
           O95819-3    undefined                 0.218474   
           P09493-3    undefined                 0.168588   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.225183   
ZYG11B     Q9C0D3      undefined                 0.252722   
ZYX        Q15942      undefined                 0.158012   
ZZEF1      O43149      undefined                 0.213065   
ZZZ3       Q8IYH5      undefined                 0.277874   

                                    autophagy_Nt_rep1_24K  \
Gene names Protein IDs Compartment                          
NaN        A0A0U1RRE5  undefined                      NaN   
           A0A0U1RRL7  undefined                      NaN   
           O95628-8    undefined                 0.258687   
           O95819-3    undefined                 0.180889   
           P09493-3    undefined                 0.144000   
...                                                   ...   
ZXDB;ZXDA  P98169      undefined                 0.229102   
ZYG11B     Q9C0D3      undefined                 0.191027   
ZYX        Q15942      undefined                 0.192820   
ZZEF1      O43149      undefined                 0.292642   
ZZZ3       Q8IYH5      undefined                 0.074801   

                                    autophagy_Nt_rep1_80K  \
Gene 

In [ ]:
#global_SVM_dict_total = {}
#global_SVM_dict = {}
#for exp in self.json_dict.keys():
##df_SVM = self.df_SVM.copy()
#    pd.read_json(self.json_dict[exp]["Misclassification Matrix"])
#    SVM_dict = {}
#    all_correct = np.diag(df_SVM)
#    members = df_SVM.sum(axis=1)
#    total_members = 0
#    membrame_members = 0
#    membrane_correct = 0
#    all_organelle_recall = []
#    all_organelle_precision = []
#    all_organelle_f1 = []
#    F1_all_cluster = []
#    no_of_membrane_clusters = 0
#    total_correct = sum(all_correct)
#    predicted_one_organelle = df_SVM.sum(axis=0)
#    
#    for i in range(len(df_SVM)):
#        total_members = total_members + members[i]
#        recall = all_correct[i]/members[i]
#        fdr = (predicted_one_organelle[i]-all_correct[i])/predicted_one_organelle[i]
#        precision = 1-fdr
#        F1 = statistics.harmonic_mean([recall, precision])
#        F1_all_cluster.append(F1)
#        SVM_dict[df_SVM["T: True group"][i]] = {"Recall": recall, "FDR": fdr, "Precision": precision, "F1": F1} 
#        if df_SVM["T: True group"][i]!="True: Nuclear pore complex" and df_SVM["T: True group"][i]!="True: Large Protein Complex" and df_SVM["T: True group"][i]!="True: Actin binding proteins" :
#            no_of_membrane_clusters = no_of_membrane_clusters+1
#            membrame_members = membrame_members + members[i]
#            membrane_correct = membrane_correct + all_correct[i]
#            all_organelle_f1.append(F1)
#            all_organelle_recall.append(recall)
#            all_organelle_precision.append(precision)
#        
#    total_recall = total_correct/total_members
#    membrane_recall = membrane_correct/membrame_members
#    av_per_organelle_recall = statistics.mean(all_organelle_recall)
#    median_per_organelle_recall = statistics.median(all_organelle_recall)
#    av_per_organelle_precision = statistics.mean(all_organelle_precision)
#    avg_organelle_f1 = statistics.mean(all_organelle_f1)
#    avg_F1_all_cluster = statistics.mean(F1_all_cluster)
#    
#    SVM_dict_total = {}
#    SVM_dict_total["Avg. all clusters"] = {"Recall": total_recall, "F1": avg_F1_all_cluster} #total recall = marker prediction accuracy
#    SVM_dict_total["Avg. all organelles"] = {"Recall": av_per_organelle_recall, "F1": avg_organelle_f1, "Precision": av_per_organelle_precision}
#    SVM_dict_total["Membrane"] = {"Recall": membrane_recall}
#    SVM_dict_total["Median. per organelle"] = {"Recall": median_per_organelle_recall}
#    
#    global_SVM_dict[exp] = SVM_dict
#    global_SVM_dict_total[exp] = SVM_dict_total
#
#    
#df_clusterPerformance_global = pd.DataFrame.from_dict({(i,j): global_SVM_dict[i][j] 
#                           for i in global_SVM_dict.keys() 
#                           for j in global_SVM_dict[i].keys()},
#                       orient='index')
#df_clusterPerformance_global.index.names = ["Experiment", "Type"]
#df_clusterPerformance_global = df_clusterPerformance_global.T 
#
#df_AvgClusterPerformance_global = pd.DataFrame.from_dict({(i,j): global_SVM_dict_total[i][j] 
#                           for i in global_SVM_dict_total.keys() 
#                           for j in global_SVM_dict_total[i].keys()},
#                       orient='index')
#df_AvgClusterPerformance_global.index.names = ["Experiment", "Type"]
#df_AvgClusterPerformance_global = df_AvgClusterPerformance_global.T

In [ ]:
df_AvgAllCluster = df_AvgClusterPerformance_global.xs("Avg. all clusters", level='Type', axis=1)
fig_markerPredictionAccuracy = go.Figure()#data=[go.Bar(x=df_test.columns, y=df_test.loc["Recall"])])
for exp in global_SVM_dict.keys():
    fig_markerPredictionAccuracy.add_trace(go.Bar(x=[exp], y=[df_AvgAllCluster[exp].loc["Recall"]]))
fig_markerPredictionAccuracy.update_layout(template="simple_white", #showlegend=False, 
            title="Cluster Performance",
            xaxis=go.layout.XAxis(linecolor="black",
                                linewidth=1,
                                mirror=True),
            yaxis=go.layout.YAxis(linecolor="black",
                                linewidth=1,
                                title="F1 score - harmonic mean of recall and precision",
                                mirror=True),
            )
fig_markerPredictionAccuracy

In [ ]:
fig_clusterPerformance = go.Figure()
list_data_type = ["Avg. all clusters", "Avg. all organelles"]
for i,exp in enumerate(global_SVM_dict.keys()):
    df_clusterPerformance = df_clusterPerformance_global.xs(exp, level='Experiment', axis=1)
    df_AvgClusterPerformance = df_AvgClusterPerformance_global.xs(exp, level='Experiment', axis=1)
    fig_clusterPerformance.add_trace(go.Scatter(x=df_clusterPerformance.columns, y=df_clusterPerformance.loc["F1"], 
                                               marker=dict(color=pio.templates["simple_white"].layout["colorway"][i]), name=exp))
    for  data_type in list_data_type:
        fig_clusterPerformance.add_trace(go.Scatter(x=[data_type], y=[df_AvgClusterPerformance[data_type].loc["F1"]],
                    mode="markers",
                    showlegend=False,
                    marker=dict(color=pio.templates["simple_white"].layout["colorway"][i])
                        ))
    fig_clusterPerformance.update_layout(template="simple_white", #showlegend=False, 
                    title="Cluster Performance",
                    xaxis=go.layout.XAxis(linecolor="black",
                                        linewidth=1,
                                        mirror=True),
                    yaxis=go.layout.YAxis(linecolor="black",
                                        linewidth=1,
                                        title="F1 score - harmonic mean of recall and precision",
                                        mirror=True),
                    )
fig_clusterPerformance

In [ ]:
#sorting_list = [ '100 min | direct DIA','100 min | library DIA', '3x 44 min | direct DIA', '3x 44 min | library DIA (8xBPRP)','3x 44 min | library DIA (3xSDB-RPS)']
df_quantity_pr_pg_combined = i_class_comp.df_quantity_pr_pg_combined.copy()    
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined.filtering != "before filtering"]
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined["Experiment"].isin(i_multi_choice.value)]
#df_quantity_pr_pg_combined = df_quantity_pr_pg_combined.assign(Experiment_lexicographic_sort = pd.Categorical(df_quantity_pr_pg_combined["Experiment"], categories=i_multi_choices, ordered=True))
#df_quantity_pr_pg_combined.sort_values("Experiment_lexicographic_sort", inplace=True)
#df_quantity_pr_pg_combined.sort_values(["Experiment_lexicographic_sort"], ascending=[True], inplace=True)
#f_quantity_pr_pg_combined.sort_values(["number of protein groups"], ascending=False, inplace=True)
layout = go.Layout(barmode="overlay",   autosize=False, width=900,height=400,  template="simple_white")
fig_quantity_pg = px.bar(df_quantity_pr_pg_combined, x="filtering", y="number of protein groups", color="type", barmode="overlay", labels={"Experiment":"", "filtering":""}, 
                         facet_col="Experiment",template="simple_white", opacity=1).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_quantity_pg.update_layout(layout, title="Number of Protein Groups")
fig_quantity_pg

In [ ]:
sorting_list = [ '100 min | direct DIA', '3x 44 min | direct DIA','100 min | library DIA', '3x 44 min | library DIA (8xBPRP)','3x 44 min | library DIA (3xSDB-RPS)']
df_cluster_normalizedMedian_ref = i_class_comp.df_cluster_normalizedMedian_ref
df_cluster_normalizedMedian_ref = df_cluster_normalizedMedian_ref.assign(Experiment_lexicographic_sort = pd.Categorical(df_cluster_normalizedMedian_ref["Experiment"], categories=sorting_list, ordered=True))
df_cluster_normalizedMedian_ref.sort_values("Experiment_lexicographic_sort", inplace=True)
fig_globalRanking = px.box(df_cluster_normalizedMedian_ref,
                                        x="Experiment",
                                        y="Normalized Median", 
                                        title="Median manhattan distance distribution for all protein clusters (n>=5 per cluster)",# "Ranking - median of all individual normalized medians - reference is the median across all experiments for each cluster",
                                        color="Experiment",
                                        points="all",
                                        template="simple_white",
                                        hover_name="Cluster", 
                        width=1000)
fig_globalRanking

In [ ]:
df_quantity_pr_pg_combined = i_class_comp.df_quantity_pr_pg_combined.copy()        
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined["Experiment"].isin(i_multi_choice.value)]
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined.assign(Experiment_lexicographic_sort = pd.Categorical(df_quantity_pr_pg_combined["Experiment"],
                                                                                                                categories=i_multi_choice.value, ordered=True))
df_quantity_pr_pg_combined.sort_values("Experiment_lexicographic_sort", inplace=True)
df_quantity_pr_pg_combined.sort_values(["Experiment_lexicographic_sort", "type"], ascending=[True, False], inplace=True)
df_quantity_pr_pg_combined.sort_values("type", ascending=False, inplace=True)

layout = go.Layout(barmode="overlay", 
  autosize=False,
  width=250*len(i_multi_choice.value),
  height=400,
  template="simple_white")

fig_quantity_pg = px.bar(df_quantity_pr_pg_combined, x="filtering", y="number of protein groups", color="type", barmode="overlay", labels={"Experiment":"", "filtering":""}, 
                         facet_col="Experiment",template="simple_white", opacity=1).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_quantity_pg.update_layout(layout, title="Number of Protein Groups")
fig_quantity_pg

In [ ]:
df_quantity_pr_pg_combined = i_class_comp.df_quantity_pr_pg_combined.copy()
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined["Experiment"].isin(i_multi_choice.value)].sort_values("filtering")
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined.type != "total"]
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined.assign(Experiment_lexicographic_sort = pd.Categorical(df_quantity_pr_pg_combined["Experiment"],
                                                                                                                categories=i_multi_choice.value, ordered=True))
#df_quantity_pr_pg_combined.sort_values("Experiment_lexicographic_sort", inplace=True)
df_quantity_pr_pg_combined.sort_values(["Experiment_lexicographic_sort", "filtering"], inplace=True)

fig_pr_dc = px.bar(df_quantity_pr_pg_combined, x="type", y="data completeness of profiles", color="filtering", barmode="overlay", labels={"Experiment":"", "type":""}, 
                         facet_col="Experiment",template="simple_white", opacity=1).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_pr_dc.update_layout(#barmode="overlay", 
                                 #xaxis_tickangle=90, 
                                 title="Data Completeness of Profiles",
                                 autosize=False,
                                 width=1800,#250*len(i_multi_choice.value),
                                 height=400,
                                 template="simple_white")
fig_pr_dc

In [ ]:
df_quantity_pr_pg_combined = i_class_comp.df_quantity_pr_pg_combined.copy()
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined[df_quantity_pr_pg_combined["Experiment"].isin(i_multi_choice.value)].sort_values("filtering")
df_quantity_pr_pg_combined = df_quantity_pr_pg_combined.assign(Experiment_lexicographic_sort = pd.Categorical(df_quantity_pr_pg_combined["Experiment"],
                                                                                                                categories=i_multi_choice.value, ordered=True))
df_quantity_pr_pg_combined.sort_values("Experiment_lexicographic_sort", inplace=True)
df_quantity_pr_pg_combined.sort_values(["Experiment_lexicographic_sort", "type"], ascending=[True, False], inplace=True)
df_quantity_pr_pg_combined.sort_values("type", ascending=False, inplace=True)

fig_pr_dc = px.bar(df_quantity_pr_pg_combined, x="type", y="data completeness of profiles", color="filtering", barmode="overlay", labels={"Experiment":"", "type":""}, 
                                 facet_col="Experiment",template="simple_white", opacity=1).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_pr_dc.update_layout(#barmode="overlay", 
                                         title="Data Completeness of Profiles",
                                         autosize=False,
                                         width=300*len(i_multi_choice.value),
                                         height=400,
                                         template="simple_white")
fig_pr_dc

In [ ]:
fg = pd.read_json(i_class_comp.json_dict["r"]["Misclassification Matrix"])
fg["T: True group"] = fg["T: True group"].str.replace(r'True: ', '')
fg

In [ ]:
.df_clusterPerformance_global

In [ ]:
#
#i_organism_comparison = pn.widgets.Select(options=list(SpatialDataSet.markerproteins_set.keys()), name="Organism", width=300) #
##i_clusterwidget_comparison = pn.widgets.Select(options=list(SpatialDataSet.markerproteins.keys()), name="Cluster of interest", width=300)
#i_clusterwidget_comparison = pn.widgets.Select(options=["Proteasome", "Lysosome"], name="Cluster of interest", width=300)
#i_clusters_for_ranking = pn.widgets.CrossSelector(name="Select clusters to be considered for ranking calculation", #value=list(SpatialDataSet.markerproteins.keys()), 
#                                                options=["Proteasome", "Lysosome"], size=8)
#i_multi_choice = pn.widgets.CrossSelector(name="Select experiments for comparison", value=["a", "b"], options=["a", "b", "c"])
#i_ref_exp = pn.widgets.Select(name="Select experiments as reference", options=["a", "b", "c"])
#i_scatter_metric = pn.widgets.Select(name="Distance metric",
#                                     options=["euclidean distance", "manhattan distance",
#                                              "1 - cosine correlation", "1 - pearson correlation"])
#i_scatter_consolidation = pn.widgets.Select(name="Consolidation of replicate distances",
#                                            options=["median","average","sum"])
#
#dashboard_json = pn.Column("Please, upload a file first and press 'Compare clusters'", name="Comparison", css_classes=["content-width"])
#comparison_status = pn.Pane("No datasets were compared yet")
#i_collapse_maps = pn.widgets.Checkbox(value=False, name="Collapse maps")
#i_collapse_cluster = pn.widgets.Checkbox(value=False, name="Collapse cluster")
#i_markerset_or_cluster = pn.widgets.Checkbox(value=False, name="Display only protein clusters")
##i_ranking_boxPlot = pn.widgets.Checkbox(value=False, name="Display box plot")
#i_ranking_boxPlot = pn.widgets.RadioBoxGroup(name="Types of ranking", options=["Box plot", "Bar plot - median", "Bar plot - sum"], inline=True)
##i_toggle_sumORmedian = pn.widgets.Toggle(name="Sum or Median", button_type="primary")
#i_ExpOverview = pn.Row(pn.Pane("", width=1000))
#wdgts_comparison = [button_comparison,i_organism_comparison] 
#
#@pn.depends(i_jsonFile.param.value)#cache_run.param.value
#def open_jsonFile(jsonFile):#run
#    cache_run_json.value = False
#    if jsonFile is None:
#        filereading_status_json = "No file is uploaded"
#        cache_uploaded_json.value = False
#        return filereading_status_json
#    else:
#        cache_uploaded_json.value = False
#        try:
#            global json_dict
#            #i_jsonFile.value
#            json_dict = ast.literal_eval(jsonFile.decode("UTF-8")) #i_class.
#            if hasattr(json_dict, "keys") == False: #i_class.
#                return "Your json-File does not fulfill the requirements"
#            else:
#                i_multi_choice.options = []
#                i_ExpOverview[0] = pn.Pane("", width=1000)
#                filereading_status_json = list(json_dict.keys())# list(set(list(SpatialDataSet.analysed_datasets_dict.keys()) + )) #i_class.
#                cache_uploaded_json.value = True
#                for wdgt in wdgts_comparison:
#                    wdgt.disabled = False
#                return pn.Column(pn.Row("You will compare following experiments:\n {}, and {}".format(", ".join(filereading_status_json[:-1]), filereading_status_json[-1])),
#                                 pn.Row(i_organism_comparison),
#                                 pn.Row(button_comparison),
#                                 )
#
#        except Exception: 
#            filereading_status_json = traceback.format_exc()
#            cache_uploaded_json.value = False
#            return filereading_status_json                 
#
#@pn.depends(i_multi_choice.param.value, i_clusterwidget_comparison.param.value, cache_run_json.param.value, i_xAxis_PCA_comp.param.value, i_yAxis_PCA_comp.param.value, 
#            i_markerset_or_cluster.param.value)
#def update_visualization_map_comparison(multi_choice, clusterwidget_comparison, run_json, xAxis_PCA_comp, yAxis_PCA_comp, markerset_or_cluster):
#    try:
#        if run_json == True:
#            if multi_choice == []:
#                return pn.Column(#pn.Row(i_multi_choice),
#                                 pn.Row("Please select experiments for comparison")
#                                )
#            else:
#                pass
#            pca_global_comparison = i_class_comp.plot_global_pca_comparison(cluster_of_interest_comparison=clusterwidget_comparison, x_PCA=xAxis_PCA_comp, y_PCA=yAxis_PCA_comp, 
#                                                                       markerset_or_cluster=markerset_or_cluster, multi_choice=multi_choice)
#            if markerset_or_cluster == False:
#                return pn.Column(#pn.Row(i_multi_choice),
#                                 pn.Row(i_clusterwidget_comparison),
#                                 pn.Row(i_markerset_or_cluster),
#                                 pn.Row(pca_global_comparison),
#                                 pn.Row(i_xAxis_PCA_comp, i_yAxis_PCA_comp)  
#                                )
#            else:
#                return pn.Column(#pn.Row(i_multi_choice),
#                                 pn.Row(i_markerset_or_cluster),
#                                 pn.Row(pca_global_comparison),
#                                 pn.Row(i_xAxis_PCA_comp, i_yAxis_PCA_comp)   
#                                )
#        else:
#            pca_global_comparison = "Run analysis first!"
#            return pca_global_comparison
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status
#
#@pn.depends((i_multi_choice.param.value)
#def update_ranking_comparison(multi_choice):
#    cluster_above_treshold = i_class_comp.distance_ranking_comparison(multi_choice=multi_choice, clusters_for_ranking=i_clusters_for_ranking.value)
#    i_clusters_for_ranking.options = list(cluster_above_treshold)
#    i_clusters_for_ranking.value = list(cluster_above_treshold)
#    
#    
#@pn.depends(i_multi_choice.param.value, i_clusterwidget_comparison.param.value, i_collapse_maps.param.value, i_collapse_cluster.param.value, i_clusters_for_ranking.param.value, 
#            i_ranking_boxPlot.param.value, cache_run_json.param.value)
#def update_distance_and_pca(multi_choice, clusterwidget_comparison, collapse_maps, collapse_cluster, clusters_for_ranking, ranking_boxPlot, run_json):
#    try:
#        if run_json == True:
#            if multi_choice == []:
#                return pn.Column(#pn.Row(i_multi_choice),
#                                 pn.Row("Please select experiments for comparison")
#                                )
#            else:
#                pass
#            #i_ref_exp.options = multi_choice
#            update_ref_exp(i_ref_exp)
#            if clusters_for_ranking == []:
#                clusters_for_ranking = [clusterwidget_comparison]
#            else:
#                pass
#            
#            
#            distance_ranking_barplot_comparison(collapse_cluster=collapse_cluster, multi_choice=i_multi_choice.value, ranking_boxPlot=ranking_boxPlot)
#            #i_clusters_for_ranking.options = list(i_class_comp.cluster_above_treshold)
#            #i_clusters_for_ranking.value = list(i_class_comp.cluster_above_treshold)
#            i_class_comp.cache_cluster_quantified = True
#            distance_comparison = i_class_comp.distance_boxplot_comparison(collapse_maps=collapse_maps, cluster_of_interest_comparison=clusterwidget_comparison, multi_choice=multi_choice)
#            if i_class_comp.cache_cluster_quantified == False:
#                return pn.Column(
#                                 pn.Row(i_clusterwidget_comparison),
#                                 pn.Row("Cluster was not quantified in any experiment")
#                                )
#            else:
#                pca_comparison = i_class_comp.plot_pca_comparison(collapse_maps=collapse_maps, cluster_of_interest_comparison=clusterwidget_comparison, multi_choice=multi_choice)
#                if clusters_for_ranking == []:
#                    return pn.Column(
#                                     pn.Row(i_clusters_for_ranking),
#                                     pn.Row("Select at least one cluster"),         
#                                     pn.Row(i_clusterwidget_comparison),
#                                     pn.Row(i_collapse_maps),
#                                     pn.Row(pca_comparison),
#                                     pn.Row(distance_comparison),
#                                     )
#                else:
#                    if i_collapse_cluster.value == True:
#                        return pn.Column(
#                                     pn.Row(i_clusters_for_ranking),
#                                     pn.Row(i_collapse_cluster, i_ranking_boxPlot),
#                                     pn.Row(distance_ranking_comparison),
#                                     #pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster, height=200, width=1800, disabled=True)),
#                                     pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster2, height=200, width=1800, disabled=True)),
#                                     pn.Row(i_clusterwidget_comparison),
#                                     pn.Row(i_collapse_maps),
#                                     pn.Row(pca_comparison),
#                                     pn.Row(distance_comparison),
#
#                                    )
#                    else:
#                        return pn.Column(
#                                     pn.Row(i_clusters_for_ranking),
#                                     pn.Row(i_collapse_cluster),
#                                     pn.Row(distance_ranking_comparison),
#                                     #pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster, height=200, width=1800, disabled=True)), 
#                                     pn.Row(pn.widgets.DataFrame(i_class_comp.df_quantified_cluster2, height=200, width=1800, disabled=True)),
#                                     pn.Row(i_clusterwidget_comparison),
#                                     pn.Row(i_collapse_maps),
#                                     pn.Row(pca_comparison),
#                                     pn.Row(distance_comparison),
#
#                                    )
#        else:
#            pca_comparison = "Run analysis first!"
#            return pca_comparison
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status
#    
#    
#@pn.depends(i_multi_choice.param.value, cache_run_json.param.value)
#def update_npr_ngg_nprDc(multi_choice, run_json):
#    try:
#        if run_json == True: 
#            if multi_choice == []:
#                return pn.Column(
#                                 pn.Row("Please select experiments for comparison"))
#            else:
#                fig_quantity_pg, fig_quantity_pr = i_class_comp.quantity_pr_pg_barplot_comparison(multi_choice=multi_choice)
#                coverage_barplot = i_class_comp.coverage_comparison(multi_choice=multi_choice)
#                return pn.Column(
#                                 pn.Row(fig_quantity_pg), 
#                                 pn.Row(fig_quantity_pr),
#                                 pn.Row(coverage_barplot)
#                                )
#        else:
#            completeness_barplot = "Run analysis first!"
#            return completeness_barplot
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status 
#    
#@pn.depends(i_multi_choice.param.value, cache_run_json.param.value)
#def update_venn(multi_choice, run_json):
#    try:
#        if run_json == True: 
#            venn_plot = []
#            if len(multi_choice)<=1:
#                return pn.Column(
#                                 pn.Row("Please select 2 or more experiments for comparison"))
#            else:
#                venn_plot, figure_UpSetPlot = i_class_comp.venn_sections(multi_choice_venn = multi_choice)
#                return pn.Column(
#                                 pn.Pane(venn_plot),
#                                 pn.Row(figure_UpSetPlot)
#                                )
#        else:
#            venn_plot = "Run analysis first!"
#            return venn_plot
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status    
#    
##i_ref_exp.options = multi_choice
#  #          update_ref_exp(i_ref_exp)
#@pn.depends(i_multi_choice.param.value, i_ref_exp.param.value, i_collapse_cluster.param.value, cache_run_json.param.value)
#def update_dynamic_range_comparison(multi_choice, ref_exp, collapse_cluster, run_json):
#    try:
#        if run_json == True: 
#            if multi_choice == []:
#                return pn.Column(
#                                 pn.Row("Please select experiments for comparison"))
#            else:
#                dynamic_range_barplot = i_class_comp.dynamic_range_comparison(collapse_cluster=collapse_cluster, multi_choice=multi_choice, ref_exp=ref_exp)
#                return pn.Column(
#                                 pn.Row(dynamic_range_barplot),
#                                 pn.Row(i_collapse_cluster),
#                                 pn.Row(i_ref_exp)
#                                )
#        else:
#            dynamic_range_barplot = "Run analysis first!"
#            return dynamic_range_barplot
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status 
#    
#    
#@pn.depends(i_multi_choice.param.value, i_scatter_metric.param.value, i_scatter_consolidation.param.value, cache_run_json.param.value)
#def update_global_scatter_comparison(multi_choice, metric, consolidation, run_json):
#    try:
#        if run_json == True: 
#            if multi_choice == []:
#                return pn.Column(#pn.Row(i_multi_choice),
#                                 pn.Row("Please select experiments for comparison"))
#            else:
#                scatter_histogram = i_class_comp.calculate_global_scatter(multi_choice=multi_choice,
#                                                                              metric=metric, consolidation=consolidation)
#                return pn.Column(pn.Row(i_scatter_metric),
#                                 pn.Row(i_scatter_consolidation),
#                                 pn.Row(scatter_histogram)
#                                )
#        else:
#            return "Run analysis first!"
#    except Exception:
#        update_status = traceback.format_exc()
#        return update_status
#    
#    
#def update_multi_choice(i_multi_choice, i_clusterwidget, i_clusters_for_ranking):
#    i_multi_choice.options = list(json_dict.keys())
#    i_clusterwidget.options = list(i_class_comp.markerproteins.keys())
#    i_clusters_for_ranking.options = list(i_class_comp.markerproteins.keys())
#    i_clusters_for_ranking.value = list(i_class_comp.markerproteins.keys())
#    i_multi_choice.value = list(json_dict.keys())
#    
#    
#@pn.depends(i_multi_choice.param.value, watch=True)
#def update_ref_exp(multi_choice):
#    i_ref_exp.options = i_multi_choice.value
#    #return i_ref_exp
#
#
#@pn.depends(i_multi_choice.param.value, watch=True)
#def update_ExpOverview(multi_choice):
#    dict_analysis_parameters={}
#    for exp_name in multi_choice:
#        dict_analysis_parameters[exp_name] = json_dict[exp_name]["Analysis parameters"]
#    i_ExpOverview[0] = pn.widgets.DataFrame(pd.DataFrame.from_dict(dict_analysis_parameters))
#    i_ExpOverview.value = pd.DataFrame.from_dict(dict_analysis_parameters)
#    i_ExpOverview.disabled = True
#    i_ExpOverview.height = 200
#
#    
#def execution_comparison(event):
#    if cache_uploaded_json.value == False:
#        comparison_status.object = "Please upload a JSON-file first"
#    else:        
#        dashboard_comparison.objects[2:] = []
#        cache_run_json.value = False
#        for wdgt in wdgts_comparison:
#            wdgt.disabled = True
#        try:
#            comparison_status.object = "Analysis in progress"
#            protein_cluster = SpatialDataSet.markerproteins_set[i_organism_comparison.value].keys()
#            update_ref_exp(i_ref_exp)
#            global i_class_comp
#            i_class_comp = SpatialDataSetComparison(clusters_for_ranking=protein_cluster, ref_exp=i_ref_exp.value, organism=i_organism_comparison.value)
#            i_class_comp.json_dict = json_dict
#            i_class_comp.read_jsonFile()
#            update_multi_choice(i_multi_choice, i_clusterwidget_comparison, i_clusters_for_ranking)
#            i_class_comp.perform_pca_comparison()
#            dashboard_comparison.append(pn.Row(i_multi_choice))
#            dashboard_comparison.append(pn.Row(i_ExpOverview))
#            dashboard_comparison.append(comparison_tabs)
#            comparison_status.object = "Comparison finished!"
#            cache_run_json.value = True
#        except Exception:
#            for wdgt in wdgts_comparison:
#                wdgt.disabled = False
#            comparison_status.object = traceback.format_exc()
#            cache_run_json.value = False
#button_comparison.on_click(execution_comparison)
#
#
#i_jsonFile_amendments_intended = pn.widgets.FileInput(name="Upload JSON file to be amended")
#i_json_ExpSelector = pn.widgets.CrossSelector(name="Select experiments, that will be removed from JSON file", width=1000)
#cache_uploaded_json_amendment = pn.widgets.Checkbox(value=False)
#cache_run_json_amendment = pn.widgets.Checkbox(value=False)
#button_reset = pn.widgets.Button(name="Reset", width=50)
#download_status = pn.Pane("Upload a JSON file first", width=1000)
#i_df_ExpComment = pn.widgets.DataFrame()
#wdgt_json = [button_reset]
#json_dict_amendments_intended = {}
##make a cache, and say, if this hasnt been executed so far, please reset it
#dict_new_expNames = {}
#dict_new_comments = {}
#    
#@pn.depends(i_jsonFile_amendments_intended.param.value)#cache_run.param.value
#def open_jsonFile_amendment(jsonFile_amendments):#run
#    cache_run_json_amendment.value = False
#    if jsonFile_amendments is None:
#        cache_uploaded_json_amendment.value = False
#    else:
#        cache_uploaded_json_amendment.value = False
#        try:
#            json_dict_cache = ast.literal_eval(i_jsonFile_amendments_intended.value.decode("UTF-8"))
#            if hasattr(json_dict_cache, "keys") == False: 
#                    return "Your json-File does not fulfill the requirements"
#            else:
#                global json_dict_amendments_intended
#                try:
#                    json_dict_amendments_intended.update(json_dict_cache)
#                except Exception:
#                    json_dict_amendments_intended = json_dict_cache
#                i_json_ExpSelector.options = list(json_dict_amendments_intended.keys())
#                cache_uploaded_json_amendment.value = True
#                for wdgt in wdgt_json:
#                    wdgt.disabled = False
#                download_status.object = "Upload sucessful"
#                return pn.Column(
#                                 i_json_ExpSelector,
#                                 i_df_ExpComment,
#                                 pn.widgets.FileDownload(callback=json_amendment_download, filename="AnalysedDatasets2.0.json"),
#                                 button_reset,
#                                 )
#        except Exception: 
#            filereading_status_json = traceback.format_exc()
#            cache_uploaded_json_amendment.value = False
#            return filereading_status_json
#
#        
#def json_amendment_download():
#    if i_json_ExpSelector.value == []:
#        download_status.object = "No experiments are selected"
#        return
#    else:
#        json_new = json_dict_amendments_intended.copy()
#        exp_names_del = [elem for elem in i_json_ExpSelector.options if not elem in i_json_ExpSelector.value]        
#        for key in exp_names_del:
#            del json_new[key]
#        checked_exp = set()
#        redundant_expNames = set(new_Exp for new_Exp in dict_new_expNames.values() if new_Exp in checked_exp or checked_exp.add(new_Exp))
#        if redundant_expNames != set():
#            download_status.object = "Experiments are not allowed to be labelled identically"
#            return
#        else:
#            for exp_name in json_new:
#                json_new[exp_name]["Analysis parameters"]["comment"] = dict_new_expNames[exp_name]
#            json_new = {dict_new_expNames[oldK]: value for oldK, value in json_new.items()}
#            sio = StringIO()
#            json.dump(
#                json_new, 
#                sio, 
#                indent=4, 
#                sort_keys=True
#            )
#            sio.seek(0)
#            download_status.object = "Download sucessful"
#            return sio
#
#        
#def reset_json_amendment(event):
#    global json_dict_amendments_intended
#    json_dict_amendments_intended = {}
#    i_json_ExpSelector.options = []
#    i_json_ExpSelector.value = []
#    i_df_ExpComment.value = pd.DataFrame()
#    for wdgt in wdgt_json:
#        wdgt.disabled = True
#    download_status.object = "Reset sucessful"
#button_reset.on_click(reset_json_amendment)
#
#
#@pn.depends(i_json_ExpSelector.param.value, watch=True)
#def update_renameExp(json_ExpSelector):
#    dict_ExpComments = {}
#    for exp_name in json_ExpSelector:
#        dict_ExpComments[exp_name] = json_dict_amendments_intended[exp_name]["Analysis parameters"]["comment"]
#    df_ExpComments = pd.DataFrame(dict_ExpComments.items(), columns=["Experiment name - old", "Comment"])#pd.DataFrame.from_dict(dict_ExpComments)
#    df_ExpComments.insert(0, "Experiment name - new", df_ExpComments["Experiment name - old"])
#    df_ExpComments.set_index("Experiment name - old", inplace=True)
#    df_ExpComments.replace({"Experiment name - new": dict_new_expNames}, inplace=True)
#    exp_previous = list(dict_new_comments.keys())
#    for exp in exp_previous:
#        if exp not in json_ExpSelector:
#            del dict_new_comments[exp]
#    df_ExpComments.loc[dict_new_comments.keys(),"Comment"] = list(dict_new_comments.values())
#    i_df_ExpComment.value = df_ExpComments
#    i_df_ExpComment.height=len(json_ExpSelector)*50
#    #return i_df_ExpComment
#
#@pn.depends(i_df_ExpComment.param.value, watch=True)
#def update_newExpNames(df_ExpComment):
#    try:        
#        global dict_new_expNames 
#        changed_expName = set(list(dict_new_expNames.values())+list(df_ExpComment["Experiment name - new"]))-set(dict_new_expNames.values())
#        dict_new_expNames = dict(zip(df_ExpComment.index, df_ExpComment["Experiment name - new"]))
#        global dict_new_comments
#        dict_new_comments_cache = dict_new_comments.copy()
#        changed_comment = set(list(dict_new_comments.values())+list(df_ExpComment["Comment"]))-set(dict_new_comments.values())
#        dict_new_comments = dict(zip(df_ExpComment.index, df_ExpComment["Comment"]))
#        if changed_expName == set() and changed_comment == set():
#            tracked_change = "No changes saved"
#        elif changed_expName != set() and changed_comment != set():
#            tracked_change = "Upload sucessful"
#        elif changed_expName != set():
#            new_exp = list(changed_expName)[0]
#            old_exp = list(dict_new_expNames.keys())[list(dict_new_expNames.values()).index(new_exp)]
#            tracked_change = "Experiment name was changed from {} to {}".format(old_exp, new_exp)
#        else:# changed_comment != set():
#            new_comment = list(changed_comment)[0]
#            exp = list(dict_new_comments.keys())[list(dict_new_comments.values()).index(new_comment)]
#            old_comment = dict_new_comments_cache[exp]
#            tracked_change = "Comment of the experiment {} was changed from {} to {}".format(exp, old_comment, new_comment)
#        download_status.object = tracked_change
#    except Exception:
#        pass
#
#
#
#
#analysis_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
#analysis_tabs.append(("Data overview", update_data_overview))
#analysis_tabs.append(("Cluster Overview", update_cluster_overview))
#analysis_tabs.append(("Cluster Details", update_cluster_details))
#analysis_tabs.append(("Dynamic Range", update_dynamic_range))
#analysis_tabs.append(("Protein Groups/Profiles Quantity", update_quantity))
#analysis_tabs.append(("Download", show_tabular_overview))
#
#comparison_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
#comparison_tabs.append(("Visualization - organellar maps", update_visualization_map_comparison))
#comparison_tabs.append(("Global Scatter", update_global_scatter_comparison))
#comparison_tabs.append(("PCA plot and Distance box plot", update_distance_and_pca))
#comparison_tabs.append(("Venn Diagram", update_venn))
#comparison_tabs.append(("Protein Groups/Profiles Quantity", update_npr_ngg_nprDc))
#comparison_tabs.append(("Dynamic Range", update_dynamic_range_comparison))
#
#
#dasboard_home = pn.Column(i_file, read_file,analysis_status, loading_status, name="Home", css_classes=["content-width"])
#dashboard_analysis = pn.Column("Please, upload a file first and press 'Analyse clusters'", name="Analysis", css_classes=["content-width"])
#dashboard_svm = pn.Column("Please, upload a file first and press 'Analyse clusters'", name="SVM Analysis", css_classes=["content-width"])
#dashboard_comparison = pn.Column(i_jsonFile, open_jsonFile, comparison_status)
#dashboard_manageDatasets = pn.Column(i_jsonFile_amendments_intended, open_jsonFile_amendment, download_status)
#
#
#app_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
#app_tabs.append(("Home", dasboard_home))
#app_tabs.append(("Analysis", dashboard_analysis))
#app_tabs.append(("SVM Analysis", dashboard_svm))
#app_tabs.append(("Data comparison", dashboard_comparison))
#app_tabs.append(("Manage Datasets", dashboard_manageDatasets))
#
#app_tabs.append(("About", pn.Row("Explanation of what's going on here", width=2000)))
#
##i_search = pn.widgets.TextInput(name="Search")
#app_center = pn.Column(pn.Row(pn.Pane("# QC tool for Spatial Proteomics", width = 600),
#                              pn.layout.HSpacer(),
#                              #i_search,
#                              #width=1600, 
#                              margin=10),
#                       app_tabs,
#                       #pn.Spacer(background="#DDDDDD", height=100, margin=0)
#                      )
#app = pn.GridSpec()#sizing_mode="stretch_both", margin=0)
#app[0,0] = pn.Spacer(background="white", margin=0) #"#DDDDDD"
#app[0,9] = pn.Spacer(background="white", margin=0) #"#DDDDDD"
#app[0,1:8] = app_center
#
#pwd = pn.widgets.PasswordInput(name="Please enter password for access.")
#app_container = pn.Column(pwd)
#
#def check_pwd(event, app=app):
#    pwd = event.new
#    if pwd == "pwd":
#        app_container[0]=app
#pwd.param.watch(check_pwd, "value")
#
#try:
#    server.stop()
#except Exception:
#    print("First server startup")
#server = app.show(port=5065, websocket_max_message_size=2000000000)
##app.servable()

In [ ]:
#df_eLifeMarkers = pd.read_csv("eLife_markers.txt", sep="\t", comment="#",
#                                       usecols=lambda x: bool(re.match("Gene name|Compartment", x)))
#df_eLifeMarkers = df_eLifeMarkers.rename(columns={"Gene name":"Gene names"})
#df_eLifeMarkers = df_eLifeMarkers.astype({"Gene names": "str"})
#i_class.df_eLifeMarkers =  df_eLifeMarkers

In [ ]:
 df_clusterPerformance = pd.DataFrame(SVM_dict)
    df_AvgClusterPerformance = pd.DataFrame(SVM_dict_total)

In [ ]:

#get_best_distribution(df_conc_fullP)


#import scipy.stats as st
#def get_best_distribution(data):
#    dist_names = ["foldnorm","genlogistic","gennorm","genpareto","genexpon","genextreme","gausshyper","gamma","gengamma","genhalflogistic","geninvgauss","gilbrat",
#"gompertz","gumbel_r","gumbel_l","halfcauchy","halflogistic","halfnorm","halfgennorm","hypsecant","invgamma","invgauss","invweibull","johnsonsb",
#"johnsonsu","kappa4","kappa3","kstwo","kstwobign","laplace","laplace_asymmetric","levy","levy_l","logistic","loggamma",
#"loglaplace","lognorm","loguniform","lomax","maxwell","mielke","moyal","nakagami","ncx2","ncf","nct","norm","norminvgauss","pareto","pearson3",
#"powerlaw","powerlognorm","powernorm","rdist","rayleigh","rice","recipinvgauss","semicircular","skewnorm","t","trapezoid","triang","truncexpon",
#"truncnorm","tukeylambda","uniform","vonmises","vonmises_line","wald","weibull_min","weibull_max", "alpha","anglit","arcsine","argus","beta","betaprime","bradford","burr",
#     "burr12","cauchy","chi","chi2","cosine","crystalball","dgamma","dweibull","erlang","expon","exponnorm","exponweib","exponpow","f","fatiguelife","fisk","foldcauchy"]
#    dist_results = []
#    params = {}
#    for dist_name in dist_names:
#        try:
#            dist = getattr(st, dist_name)
#            param = dist.fit(data)
#    
#            params[dist_name] = param
#            # Applying the Kolmogorov-Smirnov test
#            D, p = st.kstest(data, dist_name, args=param)
#            print("p value for "+dist_name+" = "+str(p))
#            dist_results.append((dist_name, p))
#        except:
#            continue
#    
#    # select the best fitted distribution
#    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
#    # store the name of the best fit and its p value
#
#    print("Best fitting distribution: "+str(best_dist))
#    print("Best p value: "+ str(best_p))
#    print("Parameters for the best fit: "+ str(params[best_dist]))
#
#    return best_dist, best_p, params[best_dist]

#import plotly.figure_factory as ff
#
#fig = ff.create_distplot(df.reset_index(), group_labels, colors=colors, bin_size=.25,
#                         show_curve=False)
#
## Add title
#fig.update_layout(title_text="Hist and Rug Plot")
#fig.show()
#

#df_cluster_normalizedMedian.index.name="Experiment"
#df_cluster_normalizedMedian.rename_axis("Cluster", axis=1, inplace=True)
#df_ranking = df_cluster_normalizedMedian.stack("Cluster")
#df_ranking.name="Normalized Median"
#df_ranking = df_ranking.reset_index()
#ranking_sum = df_cluster_normalizedMedian.sum(axis=1).round(2)
#ranking_sum.name = "Normalized Median - Sum"

#clusters_for_ranking = i_class_comp.clusters_for_ranking
#ref_exp = i_ref_exp.value
#ranking_boxPlot=True
#collapse_cluster=False
#
#df_distance_comp = i_class_comp.df_distance_comp.copy()
#df_distance_comp = df_distance_comp[df_distance_comp["Experiment"].isin(multi_choice)]
#df_distance_comp = df_distance_comp[df_distance_comp["Cluster"].isin(clusters_for_ranking)]
#
#df_quantified_cluster = df_distance_comp.reset_index()
#df_quantified_cluster = df_quantified_cluster.drop_duplicates(subset=["Cluster", "Experiment"]).set_index(["Cluster", 
#                                                                                                           "Experiment"])["distance"].unstack("Cluster")
#i_class_comp.df_quantified_cluster = df_quantified_cluster.notnull().replace({True: "x", False: "-"})
#
##dict_cluster_normalizedMedian = {}
#dict_cluster_normalizedMedian_ref = {}
#dict_median_distance_ranking = {}
#for cluster in i_class_comp.markerproteins.keys():
#    try:
#        df_cluster = df_distance_comp[df_distance_comp["Cluster"]==cluster]
#        all_median_one_cluster_several_exp = {}
#        for exp in multi_choice:
#            median = df_cluster[df_cluster["Experiment"]==exp].median()
#            all_median_one_cluster_several_exp[exp] = float(median)
#            #new
#            if exp == ref_exp:
#                ref = median
#        dict_median_distance_ranking[cluster] = all_median_one_cluster_several_exp
#        min_median = min(all_median_one_cluster_several_exp.items(), key=lambda x: x[1])[1]
#        #median_ranking = {exp: median/min_median for exp, median in all_median_one_cluster_several_exp.items()}
#        #dict_cluster_normalizedMedian[cluster] = median_ranking
#        #new
#        median_ranking_ref = {exp: median/ref[0] for exp, median in all_median_one_cluster_several_exp.items()}
#        dict_cluster_normalizedMedian_ref[cluster] = median_ranking_ref
#    except Exception:
#        continue
#
##df_cluster_normalizedMedian = pd.DataFrame(dict_cluster_normalizedMedian)
##df_cluster_normalizedMedian.index.name="Experiment"
##df_cluster_normalizedMedian.rename_axis("Cluster", axis=1, inplace=True)
##df_ranking = df_cluster_normalizedMedian.stack("Cluster")
##df_ranking.name="Normalized Median"
##df_ranking = df_ranking.reset_index()
##ranking_sum = df_cluster_normalizedMedian.sum(axis=1).round(2)
##ranking_sum.name = "Normalized Median - Sum"
##ranking_product = df_cluster_normalizedMedian.product(axis=1).round(2)
##ranking_product.name = "Normalized Median - Product"
##df_globalRanking = pd.concat([pd.DataFrame(ranking_sum), pd.DataFrame(ranking_product)], axis=1).reset_index()
##self.sorting_list = list(df_globalRanking.sort_values("Normalized Median - Sum")["Experiment"])
##set categroical column, allowing lexicographic sorting
##df_ranking = df_ranking.assign(Experiment_lexicographic_sort = pd.Categorical(df_ranking["Experiment"], categories=self.sorting_list, ordered=True))
##df_ranking.sort_values("Experiment_lexicographic_sort", inplace=True)
##df_globalRanking = df_globalRanking.assign(Experiment_lexicographic_sort = pd.Categorical(df_globalRanking["Experiment"], categories=self.sorting_list,
##                                                                                          ordered=True))
##df_globalRanking.sort_values("Experiment_lexicographic_sort", inplace=True)
##df_ranking["Experiment_lexicographic_sort"] = pd.Categorical(df_ranking["Experiment"], categories=self.sorting_list, ordered=True)
##df_ranking.sort_values("Experiment_lexicographic_sort", inplace=True)
##df_globalRanking["Experiment_lexicographic_sort"] = pd.Categorical(df_globalRanking["Experiment"], categories=self.sorting_list, ordered=True)
##df_globalRanking.sort_values("Experiment_lexicographic_sort", inplace=True)
#
#df_cluster_normalizedMedian_ref = pd.DataFrame(dict_cluster_normalizedMedian_ref)
#df_cluster_normalizedMedian_ref.index.name="Experiment"
#df_cluster_normalizedMedian_ref.rename_axis("Cluster", axis=1, inplace=True)
#
##median makes a huge differnece, improves result of DIA, MQ, libary
#df_RelDistanceRanking = pd.concat([df_cluster_normalizedMedian_ref.median(axis=1), df_cluster_normalizedMedian_ref.sem(axis=1)], axis=1, 
#                                  keys=["Distance Ranking (rel, median)", "SEM"]).reset_index().sort_values("Distance Ranking (rel, median)")
#i_class_comp.sorting_list = list(df_RelDistanceRanking["Experiment"])


#    if i_class_comp.multi_choice not in list(i_class_comp.json_dict.keys()):
#    #        i_class_comp.multi_choice = list(i_class_comp.json_dict.keys())[0:2] 
#            i_multi_choice.value = list(i_class_comp.json_dict.keys())[0:2] 
#        
#    if i_class_comp.i_clusterwidget not in list(i_class_comp.json_dict.keys()):
##        i_class_comp.multi_choice = list(i_class_comp.json_dict.keys())[0:2] 
#        i_multi_choice.value = list(i_class_comp.json_dict.keys())[0:2] 
#    
#    if i_class_comp.i_clusters_for_ranking not in list(i_class_comp.json_dict.keys()):
##    i_class_comp.multi_choice = list(i_class_comp.json_dict.keys())[0:2] 
#    i_multi_choice.value = list(i_class_comp.json_dict.keys())[0:2] 
#            
##def update_object_selector(i_mapwidget, i_clusterwidget):
##    i_mapwidget.options = list(i_class.map_names)
##    i_clusterwidget.options = list(i_class.markerproteins.keys())

    
#def update_multi_choice_venn(i_multi_choice_venn):
#    i_multi_choice_venn.options = list(json_dict.keys())
    #i_multi_choice_venn.value = list(json_dict.keys())[0:2]
    
    #if i_class_comp.multi_choice_venn not in list(i_class_comp.json_dict.keys()):
    #        i_class_comp.multi_choice_venn = list(i_class_comp.json_dict.keys())[0:2]  
    #        i_multi_choice_venn.value = list(i_class_comp.json_dict.keys())[0:2] 
     
        
#                   i_renameExp.disabled =  False
                    #dict_old_new_ExpName = dict.fromkeys(json_dict_amendments_intended.keys(), "")
#                    i_renameExp.disabled =  True
                    #dict_old_new_ExpName = dict.fromkeys(json_dict_amendments_intended.keys(), "")
                #filereading_status_json = list(json_dict_amendments_intended.keys())
              
    #        for key in dict_old_new_ExpName.keys():
    #            if dict_old_new_ExpName[key] == "":
    #                dict_old_new_ExpName[key] = key  
    
#    i_renameExp.disabled = False
#    i_renameExp.options = json_ExpSelector
#    return i_renameExp 
#
#@pn.depends(i_renameExp_text.param.value, watch=True)
#def update_dict_expName(renameExp_text):
#    dict_old_new_ExpName[i_renameExp.value] = renameExp_text 
#
#@pn.depends(i_renameExp.param.value, watch=True)
#def update_text(renameExp):
#    i_renameExp_text.value = dict_old_new_ExpName[renameExp]





####SILAC
###df_data_completeness = i_class.df_index["Ratio H/L"].stack(["Fraction"])
###data_completeness = 1-df_data_completeness.apply(np.isnan).apply(sum)/len(df_data_completeness)
###data_completeness = data_completeness.append(pd.Series(data_completeness.mean(), index=["combined"]))
###data_completeness.rename("Data completness", inplace=True)
####LFQ
###df_data_completeness = i_class.df_index["LFQ intensity"].stack(["Fraction"])
###data_completeness = 1-df_data_completeness.apply(np.isnan).apply(sum)/len(df_data_completeness)
###data_completeness = data_completeness.append(pd.Series(data_completeness.mean(), index=["combined"]))
###data_completeness.rename("Data completness", inplace=True)
####LFQ and SILAC
###df_profile_completeness = i_class.df_01_stacked["normalized profile"].xs(i_class.fractions[0], level="Fraction", axis=0).unstack(["Map"])
###profile_completeness = 1-df_profile_completeness.apply(np.isnan).apply(sum)/len(df_profile_completeness)
###profile_completeness = profile_completeness.append(pd.Series(profile_completeness.mean(), index=["combined"]))
###profile_completeness.rename("Profile completness", inplace=True)





###
###df_completeness = pd.concat([data_completeness, profile_completeness], axis=1)
###df_completeness.index.name = "Map"###################
#SVM
###################

#df_01_test = i_class.df_01_stacked.copy()
#
##df_01_filtered_combined = df_01_filtered_combined.stack(["Experiment", "Map"]).swaplevel(0,1, axis=0).dropna(axis=1)
#index_ExpMap = df_01_test.index.get_level_values("Map")+"_"+df_01_test.index.get_level_values("Fraction")
#index_ExpMap.name = "Map_Frac"
#df_01_test.set_index(index_ExpMap, append=True, inplace=True)      
#
#df_01_test.index = df_01_test.index.droplevel(["Map", "Fraction"])
#df_01_test = df_01_test.unstack("Map_Frac")
#df_01_test.columns = ["_".join(col) for col in df_01_test.columns.values]
#df_01_test.rename(index={"undefined" : np.nan}, level="Compartment", inplace=True)
#df_01_test
#compartments = i_class.df_eLifeMarkers["Compartment"].unique()
#compartment_color = dict(zip(compartments, i_class.css_color))
#compartment_color["undefined"] = "lightgrey"
#
#collapse_maps_PCA_test = False
#df_global_pca = i_class.df_pca.unstack("Map").swaplevel(0,1, axis=1)[i_class.map_of_interest].reset_index()
##        else:
# #           df_global_pca = self.df_pca.rename_axis("PC", axis="columns").unstack("Map").stack("PC").median(axis=1).to_frame().unstack("PC")
# #           df_global_pca.columns = df_global_pca.columns.droplevel()
  #          df_global_pca.reset_index(inplace=True)

#if collapse_maps_PCA_test == False:
#    title_PCA = "Protein subcellular localization by PCA of {}".format(i_class.map_of_interest)
#else:
#    title_PCA = "Protein subcellular localization by PCA of combined maps"
        
#fig_global_pca = px.scatter(data_frame=df_global_pca,
#                            x="PC1",
#                            y="PC3",
#                            color="Compartment",
#                            color_discrete_map=compartment_color,
#                            title= "Protein subcellular localization by PCA for {}".format(i_class.map_of_interest) 
#                            if collapse_maps_PCA_test == False else "Protein subcellular localization by PCA of combined maps", 
#                            #title_PCA,
#                            hover_data=["Protein IDs", "Gene names", "Compartment"],
#                    #       custom_data=df_annotated_all.columns,
#                            opacity=0.9
#                            )
#fig_global_pca       
######################################################WITH RNAKING 
##dict_cluster_ranking = {}
##dict_cluster_normalizedMedian = {}
##for cluster in i_class.markerproteins.keys():
##    df_cluster = df_distance_map_cluster_gene_in_index.xs(cluster, level="Cluster")
##    all_median_one_cluster_several_exp = {}
##    for exp in i_class.json_dict.keys():
##        median = df_cluster.xs(exp, level="Experiment").median()
##        #df_cluster[df_cluster.index.get_level_values("Experiment").isin([exp])].median()
##        all_median_one_cluster_several_exp[exp] = float(median)
##    
##    min_median = min(all_median_one_cluster_several_exp.items(), key=lambda x: x[1])[1]
##    median_ranking = {exp: median/min_median for exp, median in all_median_one_cluster_several_exp.items()}
##    dict_cluster_normalizedMedian[cluster] = median_ranking
##    
##    dict_cluster_normalizedMedian[cluster] = median_ranking
##    
##    ranking_one_cluster = {key: rank for rank, key in enumerate(sorted(all_median_one_cluster_several_exp, key=all_median_one_cluster_several_exp.get), 1)}
##    dict_cluster_ranking[cluster] = ranking_one_cluster
##
##df_ranking = pd.concat([pd.DataFrame(dict_cluster_ranking), pd.DataFrame(dict_cluster_normalizedMedian)], keys=["Ranking", "Normalized median"], names=["Type", "Experiment"])   
##df_ranking.rename_axis("Cluster", axis=1, inplace=True)
##df_rank_for_plot = df_ranking.unstack("Type").stack("Cluster").reset_index()
##df_rank_for_plot = df_rank_for_plot.sort_values(["Normalized median", "Experiment"])
##fig2 =px.bar(df_rank_for_plot, x="Cluster", y="Normalized median", color="Experiment", barmode="group")
##fig2.update_xaxes(categoryorder="total ascending")
##fig2 

##ranking_sum = df_ranking.sum(axis=1).round(2)
##df_ranking2 = df_ranking.copy()
##df_ranking2["Ranking - Product"] = df_ranking2.product(axis=1).round(2)
##df_ranking2["Ranking - Sum"] = ranking_sum
##df_ranking2

#on_validProfiles2 = {}
#or maps in i_class.map_names : 
#   series_non_validProfiles_map = df_MQ["normalized profile"].xs(maps, level="Map").unstack("Fraction").apply(lambda x: x.isnull().any(), axis=1)
#   total_protein_ids_map = len(series_non_validProfiles_map)
#   non_validProfiles_map = len(series_non_validProfiles_map[series_non_validProfiles_map == True].index)
#   non_validProfiles2[maps] = non_validProfiles_map/total_protein_ids_map*100
#eries_non_validProfiles_combined = df_MQ.unstack(["Fraction", "Map"])["normalized profile"].apply(lambda x: x.isnull().any(), axis=1)
#otal_protein_ids_combined = len(series_non_validProfiles_combined)
#on_validProfiles_combined = len(series_non_validProfiles_combined[series_non_validProfiles_combined == True].index)
#on_validProfiles2["combined maps"] = non_validProfiles_combined/total_protein_ids_combined*100
#self.shape_dict["Non valid profiles"] = non_validProfiles
#on_validProfiles2


##df_lognorm_MAP1 = df_lognorm_ratio_stacked.loc(axis=0)[:,:,:,:,:,:,:,:,["MAP1"]]
##px.histogram(df_lognorm_MAP1.reset_index(), 
##x="normalized profile",
##color="Map_Frac",
##facet_row="Map_Frac")
#
#timsTOF = i_class.df_01_stacked.copy()
#timsTOF_tojson = i_class.df_01_stacked["normalized profile"].unstack("Map").median(axis=1).to_frame(name="normalized profile")#.unstack("PC")
#
#
#test_summary_dict = {"timsTOF" : {"0/1 normalized data" : timsTOF_tojson.reset_index().to_json() }}
#test_summary_dict["Expl"] = {"0/1 normalized data" : Expl_tojson.reset_index().to_json() }
#
#for exp_name in test_summary_dict.keys():
#    #print(exp_name)
#    for data_type in test_summary_dict[exp_name].keys():
#        if data_type == "0/1 normalized data" and exp_name == list(test_summary_dict.keys())[0]:
#            #convert into dataframe
#            df_01_combined = pd.read_json(test_summary_dict[exp_name][data_type])
#            #get only 01 normalized data 
#            df_01_combined = df_01_combined.set_index(["Fraction", "Gene names", "Protein IDs", "Compartment"])[["normalized profile"]].unstack(["Fraction"])
#            df_01_combined.rename(columns = {"normalized profile":exp_name}, inplace=True)
#
#        elif data_type == "0/1 normalized data" and exp_name != list(test_summary_dict.keys())[0]:
#            df_01_toadd = pd.read_json(test_summary_dict[exp_name][data_type])
#            df_01_toadd = df_01_toadd.set_index(["Fraction", "Gene names", "Protein IDs", "Compartment"])[["normalized profile"]].unstack(["Fraction"])
#            df_01_toadd.rename(columns = {"normalized profile":exp_name}, inplace=True)
#            #dataframes will be concatenated, only proteins/Profiles that are in both df will be retained
#            df_01_combined = pd.concat([df_01_combined, df_01_toadd], axis=1, join="inner")
#            
#            
#df_01_filtered_combined = df_01_combined.dropna()    
#df_01_filtered_combined.columns.names = ["Experiment", "Fraction"]
##reframe it to make it ready for PCA
#df_01_filtered_combined = df_01_filtered_combined.stack(["Experiment"])
#
#
#pca = PCA(n_components=3)
#
## df_pca: PCA processed dataframe, containing the columns "PC1", "PC2", "PC3"
#df_pca = pd.DataFrame(pca.fit_transform(df_01_filtered_combined))
#df_pca.columns = ["PC1", "PC2", "PC3"]
#df_pca.index = df_01_filtered_combined.index
#
#
#
#df_pca_for_plotting = i_class.df_global_pca_for_plotting
#df_pca_for_plotting
#
#
#fig_pca = px.scatter(data_frame=i_class.df_global_pca_for_plotting.reset_index(),
#                     x="PC1",
#                     y="PC3",
#                     color="Compartment",
#                     color_discrete_map={
#                         "undefined": "lightgrey",
#                         "Endosome": "green",
#                         "Ergic/cisGolgi": "blue",
#                         "Large Protein Complex": "orange",
#                         "Nuclear pore complex": "purple",
#                         "Peroxisome" : "goldenrod",
#                         "Lysosome": "yellow",
#                         "Plasma membrane" : "lightcoral",
#                         "Actin binding proteins" : "magenta",
#                         "ER": "brown",
#                         "ER_high_curvature" : "lightpink",
#                         "Golgi": "red",
#                         "Mitochondrion": "turquoise",
#                     },
#                     facet_col="Experiment",
#                     #facet_col_wrap=4,
##  facet_col_wrap=len(i_class.fractions),     
#                         
#                     )
#fig_pca
#
#
##compartments = self.df_eLifeMarkers["Compartment"].unique()
##compartment_color = dict(zip(compartments, css_color))
##compartment_color["undefined"] = "lightgrey"
#
#fig_pca = px.scatter(data_frame=i_class.df_pca.reset_index(),
#                     x="PC1",
#                     y="PC3",
#                     color="Compartment",
#                     color_discrete_map=compartment_color,
#                    title="Protein subcellular localization by PCA",
#                    hover_data=["Gene names", "Compartment"], #"Protein names", 
#                    #custom_data=df_annotated_all.columns,
#                    opacity=0.9
#                    )
#fig_pca                         

#df_fractionnumber_stacked = i_class.df_index.copy().stack("Fraction")
#number_fractions = len(df_fractionnumber_stacked.index.get_level_values("Fraction").unique())
#
#df_index = i_class.df_index.stack("Map")
#df_index.sort_index(axis=1, level=0, inplace=True)
#df_mscount_mapstacked = df_index.loc[df_index[("MS/MS count")].apply(np.sum, axis=1) >= (
#        number_fractions * 2)]
#df_stringency_mapfracstacked = df_mscount_mapstacked.copy()
#df_stringency_mapfracstacked = df_stringency_mapfracstacked.loc[
#    df_stringency_mapfracstacked[("LFQ intensity")].apply(lambda x: any(
#        np.invert(np.isnan(x)).rolling(window=4).sum() >=
#        4), axis=1)]
#df_stringency_mapfracstacked = df_stringency_mapfracstacked.copy().stack("Fraction")
#
#df_stringency_mapfracstacked

#df_eLifeMarkers = pd.read_csv("eLife_markers.txt", sep="\t", comment="#",
#                                       usecols=lambda x: bool(re.match("Gene name|Compartment", x)))
#df_eLifeMarkers = df_eLifeMarkers.rename(columns={"Gene name":"Gene names"})
#df_eLifeMarkers = df_eLifeMarkers.astype({"Gene names": "str"})
#df_eLifeMarkers

#df_index_annotated_noIndex = df_stringency_mapfracstacked.reset_index()
#df_index_annotated_noIndex = df_index_annotated_noIndex.merge(df_eLifeMarkers, how="outer", on="Gene names", indicator = True)
##df_index_annotated_noIndex = df_index_annotated_noIndex.sort_values("Compartment")
#df_an = df_index_annotated_noIndex.loc[df_index_annotated_noIndex["_merge"].isin(["both","left_only"])].drop("_merge", axis=1)
#df_an.set_index([c for c in df_an.columns if c != "MS/MS count" and c != "LFQ intensity"], inplace=True)
#df_an.rename(index={np.nan : "undefined"}, level="Compartment", inplace=True)
#df_an

#df_pca_MAP1 = i_class.df_pca.unstack("Map").swaplevel(0,1, axis=1)["Map2"].reset_index()
#df_pca_MAP1

#fig_pca = px.scatter(data_frame=df_pca_MAP1,
#                     x="PC1",
#                     y="PC3",
#                     color="Compartment",
#                     color_discrete_map=compartment_color,
#                    title="Protein subcellular localization by PCA",
#                    hover_data=["Gene names", "Compartment"], #"Protein names", 
#                    #custom_data=df_annotated_all.columns,
#                    opacity=0.9
#                    )
#fig_pca  


#i_class.df_pca.query("Compartment == "ER" or Compartment == "Plasma membrane"")#"& Compartment == "Plasma memebrane"")  and 

#json_dict = ast.literal_eval(i_jsonFile.value.decode("UTF-8"))



#    @pn.depends(i_SVM_table.param.value, cache_run.param.value)
#def read_SVM_matrix(SVM_table, run):   
#    try:
#        if run == True:
#            if SVM_table == "":
#                SVM_reading_status = "No misclassification matrix is uploaded"
#                cache_uploaded_SVM.value = False
#            else:
#                cache_uploaded_SVM.value = False
#                try:
#                    i_class.df_SVM = pd.read_table(StringIO(SVM_table), sep="\t")
#                    SVM_reading_status = i_class.df_SVM
#                    cache_uploaded_SVM.value = True
#                    return pn.Column(pn.Pane(SVM_reading_status, width=60*SVM_reading_status.shape[1]),
#                                     pn.Row(button_SVM_analysis))
#                except: 
#                    SVM_reading_status = traceback.format_exc()
#                    cache_uploaded_SVM.value = False
#                    return SVM_reading_status 
#        else:
#            SVM_reading_status = "Please, upload a file first and press ‘Analyse clusters’"
#            return SVM_reading_status
#    except:
#        SVM_reading_status = traceback.format_exc()
#        return SVM_reading_status#        


#from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
#from matplotlib_venn import venn3, venn3_circles
#from matplotlib import pyplot as plt
#Course1=["A", "B","C", "D", "E", "F", "G", "I", "P", "Q"]
#Course2=["B", "E", "F", "H","K", "Q", "R", "S", "T", "U", "V", "Z"]
#Course3=["C", "E", "G", "H", "J", "K", "O", "Q", "Z"]
#
#vd3=venn3([set(Course1),set(Course2),set(Course3)],
# set_labels=("Course1", "Course2","Course3"),
# set_colors=(), 
# alpha = 0.8)
#venn3_circles([set(Course1), set(Course2),set(Course3)], linestyle="-.", linewidth=2, color="grey")
#for text in vd3.set_labels:
#    text.set_fontsize(16);
#for text in vd3.subset_labels:
#    text.set_fontsize(16)
##plt.title(‘Venn Diagram for 3 courses’,fontname=’Times New Roman’,fontweight=’bold’,fontsize=20,
## pad=30,backgroundcolor=’#cbe7e3",color=’black’,style=’italic’);
#plt.show()

#                    data_completeness: series, for each individual map, as well as combined maps: 1 - (percentage of NANs)

            ##profile completness - percentage of valid profiles (profiles that do not contain NaN)
            #df_profile_completeness = df_01_stacked["normalized profile"].xs(self.fractions[0], level="Fraction", axis=0).unstack(["Map"])
            #profile_completeness = 1-df_profile_completeness.apply(np.isnan).apply(sum)/len(df_profile_completeness)
            #profile_completeness = profile_completeness.append(pd.Series(profile_completeness.mean(), index=["Combined Maps"]))
            #profile_completeness.rename("Profile completeness", inplace=True)
            #
            #df_completeness = pd.concat([self.data_completeness, profile_completeness], axis=1)
            #df_completeness.index.name = "Map"
            #self.analysis_summary_dict["Data/Profile Completeness"] = df_completeness.reset_index().to_json()
            
            
            
                        ##profile completness - percentage of valid profiles (profiles that do not contain NaN)
            #df_profile_completeness = df_01_stacked["normalized profile"].xs(self.fractions[0], level="Fraction", axis=0).unstack(["Map"])
            #profile_completeness = 1-df_profile_completeness.apply(np.isnan).apply(sum)/len(df_profile_completeness)
            #profile_completeness = profile_completeness.append(pd.Series(profile_completeness.mean(), index=["Combined Maps"]))
            #profile_completeness.rename("Profile completeness", inplace=True)
            #            
            #df_completeness = pd.concat([self.data_completeness, profile_completeness], axis=1)
            #df_completeness.index.name = "Map"
            #self.analysis_summary_dict["Data/Profile Completeness"] = df_completeness.reset_index().to_json()
            #
#df_completeness_combined : df, with information about Data/Profile Completeness, index: "Experiment", "Map", 
#                                       column names: "Data completeness", "Profile completeness"


#                    data_completeness : series, for each individual map, as well as combined maps: 1 - (percentage of NANs)
#            #data completness - percentage of NANs
#            if self.acquisition == "SILAC":
#                df_data_completeness = df_index["Ratio H/L"].stack(["Fraction"])
#            elif self.acquisition == "LFQ":
#                df_data_completeness = df_index["LFQ intensity"].stack(["Fraction"])
#                
#            data_completeness = 1-df_data_completeness.apply(np.isnan).apply(sum)/len(df_data_completeness)
#            data_completeness = data_completeness.append(pd.Series(data_completeness.mean(), index=["Combined Maps"]))
#            data_completeness.rename("Data completeness", inplace=True)
#            self.data_completeness = data_completeness 



#                    data_completeness : series, for each individual map, as well as combined maps: 1 - (percentage of NANs)
            
#            #data completness - percentage of NANs
#            df_data_completeness = df_index["LFQ intensity"].stack(["Fraction"])
#            data_completeness = 1-df_data_completeness.apply(np.isnan).apply(sum)/len(df_data_completeness)
#            data_completeness = data_completeness.append(pd.Series(data_completeness.mean(), index=["Combined Maps"]))
#            data_completeness.rename("Data completeness", inplace=True)
#            self.data_completeness = data_completeness


                
#                self:
#                    analysis_summary_dict["Data/Profile Completeness"] : df, with information about Data/Profile Completeness
#                                        column: "Experiment" 	"Map" 	"Data completeness" 	"Profile completeness"
#                                        no row index

In [ ]:
SVM_dict_total

In [ ]:
try:
    x = s['mainsnak']['datavalue']['value']['numeric-id']
except KeyError:
    pass


In [ ]:
d = {"a": {"b": {"c"}}}
d.get("a", {}).get("b", {})

In [ ]:
i_SVM_table.value = {"A":"b"}

In [ ]:
i_multi_choice.value

In [ ]:
i_json_ExpSelector.value

In [ ]:
df_SVM.set_index("T: True group")[::-1]

In [ ]:
df_SVM = pd.read_table(StringIO(i_SVM_table.value), sep="\t")

In [ ]:
xx.value

In [ ]:
xx = pn.widgets.input.TextAreaInput(name="Misclassification matrix from Perseus", placeholder="Copy matrix here...")

In [ ]:
i_SVM_table.value# = pn.widgets.input.TextAreaInput(name="Misclassification matrix from Perseus", placeholder="Copy matrix here...")

In [ ]:
json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"]

In [ ]:
pd.read_json(json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"])

In [ ]:
i_SVM_table.value = json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"]

In [ ]:
str(json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"])

In [ ]:
(i_SVM_table.value)

In [ ]:
pd.read_json(df_SVM)

In [ ]:
df_SVM = pd.read_json(json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"])

In [ ]:
json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"]

In [ ]:
json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"]

In [ ]:
pd.read_json(json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"])

In [ ]:
json_dict_amendments_intended[i_exp_SVM.value]["Misclassification Matrix"]#.

In [ ]:
if hasattr(i_SVM_table.value, "keys") == True: 
    print("NO")
else:
    print("True")


In [ ]:
dictio["a"]

In [ ]:
dictio = {"a":"gb"}